In [ ]:
enhancers = pd.read_excel("/content/drive/MyDrive/New_run_with_additional_features/BED files/Genomic analyses/prioritized enhancers.xlsx", skiprows = 1, skipfooter=2)
enhancers = enhancers[['Chromosome', 'Start', 'End']]
enhancers

,Chromosome,Start,End
0,chr1,950385,951076
1,chr1,26601221,26605050
2,chr11,60808583,60810623
3,chr11,60905509,60908833
4,chr11,60916775,60922405
...,...,...,...
33104,chr11,69313428,69315634
33105,chr11,69452609,69454414
33106,chr11,70327981,70329951
33107,chr1,28498566,28501354


In [ ]:
def export_to_bed_files(df, filename):
  df.to_csv(filename, sep='\t', index=False, header=False)

export_to_bed_files(enhancers, filename = "/content/drive/MyDrive/New_run_with_additional_features/BED files/Genomic analyses/unlifted_enhancers.txt")


In [ ]:
enhancers = pd.read_csv("/content/drive/MyDrive/New_run_with_additional_features/BED files/Genomic analyses/lifted_enhancers.bed", sep = '\t', header = None)
enhancers

,0,1,2,3,4
0,chr1,1015005,1015696,chr1:950386-951076,1
1,chr1,26274730,26278559,chr1:26601222-26605050,1
2,chr11,61041111,61043151,chr11:60808584-60810623,1
3,chr11,61138037,61141361,chr11:60905510-60908833,1
4,chr11,61149303,61154933,chr11:60916776-60922405,1
...,...,...,...,...,...
33101,chr11,69498660,69500866,chr11:69313429-69315634,1
33102,chr11,69637841,69639646,chr11:69452610-69454414,1
33103,chr11,70481876,70483846,chr11:70327982-70329951,1
33104,chr1,28172055,28174843,chr1:28498567-28501354,1


In [ ]:
enhancers

,0,1,2,3,4
0,chr1,1015005,1015696,chr1:950386-951076,1
1,chr1,26274730,26278559,chr1:26601222-26605050,1
2,chr11,61041111,61043151,chr11:60808584-60810623,1
3,chr11,61138037,61141361,chr11:60905510-60908833,1
4,chr11,61149303,61154933,chr11:60916776-60922405,1
...,...,...,...,...,...
33101,chr11,69498660,69500866,chr11:69313429-69315634,1
33102,chr11,69637841,69639646,chr11:69452610-69454414,1
33103,chr11,70481876,70483846,chr11:70327982-70329951,1
33104,chr1,28172055,28174843,chr1:28498567-28501354,1


In [ ]:
import subprocess

# official code to run and install the bed files, within a folder
mastersheet = "/content/drive/MyDrive/New_run_with_additional_features/mastersheet/2023_07_21_master_SCREEN_zebrafirsh_ncer_tfbs_dickelscore_vistaheartenhancers_cvdkp_hic_pchic_atac.txt"

def read_cres_files(file_lst, folder, subject, header):
  for bed_file in file_lst:
    try:
      df = pd.read_csv(os.path.join(folder, bed_file), sep = '\t', header = header)
    except pd.errors.EmptyDataError:
      # handle empty file
      print(f"One missing file: {bed_file}")
      pass
    else:
      name = bed_file[: -4]
      # print(df)
      # print(df['name'].value_counts())
      # print(bed_file)
      output_file = f"/content/drive/MyDrive/New_run_with_additional_features/BED files merging test/{subject}/intersect_{name}.txt"

      # Run the Bedtools intersect command
      command = ['bedtools', 'intersect', '-C', '-a', mastersheet, '-b', os.path.join(folder, bed_file)]
      result = subprocess.run(command, capture_output=True, text=True)

      # Save the intersected result to the output file
      with open(output_file, 'w') as f:
          f.write(result.stdout)

      # Display the output file name
      print(f"Intersection saved as {output_file}")


def read_tfbs_files(subject, header = None):
  folder = f"/content/drive/MyDrive/New_run_with_additional_features/BED files/{subject}"

  files = [ele for ele in os.listdir(folder)]
  # files = [ele for ele in os.listdir(folder) if 'unmapped' not in ele]

  print(len(files), files)

  read_cres_files(files, folder, subject, header)

read_tfbs_files("Genomic analyses clean fetal heart enhancers")

2 ['lifted_human_fetal_heart_enhancers_prioritized_fetal_heart_chromatin.bed', 'unlifted_enhancers.txt']
Intersection saved as /content/drive/MyDrive/New_run_with_additional_features/BED files merging test/Genomic analyses clean fetal heart enhancers/intersect_lifted_human_fetal_heart_enhancers_prioritized_fetal_heart_chromatin.txt
Intersection saved as /content/drive/MyDrive/New_run_with_additional_features/BED files merging test/Genomic analyses clean fetal heart enhancers/intersect_unlifted_enhancers.txt


In [ ]:
sample_intersect = pd.read_csv("/content/drive/MyDrive/New_run_with_additional_features/BED files merging test/Genomic analyses clean fetal heart enhancers/intersect_unlifted_enhancers.txt", sep = '\t', header = None)
sample_intersect[222].value_counts()

<ipython-input-34-7d6487ac400a>:1: DtypeWarning: Columns (27,72) have mixed types. Specify dtype option on import or set low_memory=False.
  sample_intersect = pd.read_csv("/content/drive/MyDrive/New_run_with_additional_features/BED files merging test/Genomic analyses clean fetal heart enhancers/intersect_unlifted_enhancers.txt", sep = '\t', header = None)


0     23307
1      1390
8        16
2         7
16        4
4         1
Name: 222, dtype: int64

In [ ]:
# quality check for mastersheet before merging and run merging using bedtools
# everything was already at hg38, there is no need to lift over anymore, use unlifted version
mastersheet = "/content/drive/MyDrive/New_run_with_additional_features/mastersheet/2023_07_21_master_SCREEN_zebrafirsh_ncer_tfbs_dickelscore_vistaheartenhancers_cvdkp_hic_pchic_atac.txt"

def merge_based_on_3_cols(df1, df2, target_col, col_index):
  df2 = df2.rename(columns = {0:'chrom', 1: 'POS', 2: "POS_INCR", col_index: target_col, 3: "ID"})[['chrom', 'POS', 'POS_INCR', target_col, 'ID']]
  return pd.merge(df1, df2, on = ['chrom', 'POS', 'POS_INCR', 'ID'])

# merge() arguments: mastersheet and subject
def merge(subject, mastersheet, header = None):
  master = pd.read_csv(mastersheet, sep = '\t')
  folder = f"/content/drive/MyDrive/New_run_with_additional_features/BED files merging test/{subject}"
  print(len(os.listdir(folder)))
  for file in os.listdir(folder):
    print(file)
    df = pd.read_csv(os.path.join(folder, file), sep = '\t', header = None)
    # print(df)
    print(df[df.columns[-1]].value_counts())
    # master[file[10:-4]] = df[df.columns[-1]]
    # print(df)
    master = merge_based_on_3_cols(master, df, file[10:-4], df.columns[-1])
    # print(master[master.columns[-1]].value_counts())
    # break


  return master

master = merge("Genomic analyses clean fetal heart enhancers", mastersheet)
master

<ipython-input-51-690f51b5a882>:11: DtypeWarning: Columns (27,72) have mixed types. Specify dtype option on import or set low_memory=False.
  master = pd.read_csv(mastersheet, sep = '\t')


1
intersect_unlifted_fetal_heart_enhancers.txt
0     23307
1      1390
8        16
2         7
16        4
4         1
Name: 222, dtype: int64


<ipython-input-51-690f51b5a882>:16: DtypeWarning: Columns (27,72) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(folder, file), sep = '\t', header = None)


,chrom,POS,POS_INCR,ID,REF,ALT,QUAL,FILTER,ALLELEID,AF_ESP,...,ENCFF783MYY,ENCFF731JBR,ENCFF988KSK,ENCFF984UBD,ENCFF968YWL,ENCFF953FJE,ENCFF901EKY,ENCFF898WYV,ENCFF991MKU,unlifted_fetal_heart_enhancers
0,chr1,1013983,1013984,1028857,G,A,.,.,1015460,.,...,4,4,4,4,3,4,4,4,4,0
1,chr1,1043733,1043734,2063004,G,T,.,.,2124407,.,...,0,0,0,0,0,0,0,0,0,0
2,chr1,1043822,1043823,2020643,G,A,.,.,2079827,.,...,0,0,0,0,0,0,0,0,0,0
3,chr1,1045278,1045279,2161460,G,T,.,.,1887239,.,...,0,0,0,0,0,0,0,0,0,0
4,chr1,1046736,1046737,1525296,G,A,.,.,985908,.,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24720,chrX,154965968,154965969,1698786,A,C,.,.,1691162,.,...,0,0,0,0,0,0,0,0,0,0
24721,chrX,154966425,154966426,811704,C,T,.,.,800291,.,...,0,0,0,0,0,0,0,0,0,0
24722,chrX,154984682,154984683,1330594,C,G,.,.,1321273,.,...,0,0,0,0,0,0,0,0,0,0
24723,chrX,154984804,154984805,1698780,C,T,.,.,1691156,.,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
master['unlifted_fetal_heart_enhancers'].value_counts()

0     23307
1      1390
8        16
2         7
16        4
4         1
Name: unlifted_fetal_heart_enhancers, dtype: int64

In [ ]:
master

,chrom,POS,POS_INCR,ID,REF,ALT,QUAL,FILTER,ALLELEID,AF_ESP,...,ENCFF783MYY,ENCFF731JBR,ENCFF988KSK,ENCFF984UBD,ENCFF968YWL,ENCFF953FJE,ENCFF901EKY,ENCFF898WYV,ENCFF991MKU,unlifted_fetal_heart_enhancers
0,chr1,1013983,1013984,1028857,G,A,.,.,1015460,.,...,4,4,4,4,3,4,4,4,4,0
1,chr1,1043733,1043734,2063004,G,T,.,.,2124407,.,...,0,0,0,0,0,0,0,0,0,0
2,chr1,1043822,1043823,2020643,G,A,.,.,2079827,.,...,0,0,0,0,0,0,0,0,0,0
3,chr1,1045278,1045279,2161460,G,T,.,.,1887239,.,...,0,0,0,0,0,0,0,0,0,0
4,chr1,1046736,1046737,1525296,G,A,.,.,985908,.,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24720,chrX,154965968,154965969,1698786,A,C,.,.,1691162,.,...,0,0,0,0,0,0,0,0,0,0
24721,chrX,154966425,154966426,811704,C,T,.,.,800291,.,...,0,0,0,0,0,0,0,0,0,0
24722,chrX,154984682,154984683,1330594,C,G,.,.,1321273,.,...,0,0,0,0,0,0,0,0,0,0
24723,chrX,154984804,154984805,1698780,C,T,.,.,1691156,.,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
from datetime import date
def export_mastersheet(df, note):
  today = date.today()
  import re
  today = re.sub(r'-', '_', str(today))
  df.to_csv(f"mastersheet/{today}_master_{note}.txt", index = False, sep = '\t')
  print(today)

export_mastersheet(master, note = 'SCREEN_zebrafirsh_ncer_tfbs_dickelscore_vistaheartenhancers_cvdkp_hic_pchic_atac_fetalheartenhancers')

2023_07_28


In [ ]:
master

,chrom,POS,POS_INCR,ID,REF,ALT,QUAL,FILTER,ALLELEID,AF_ESP,...,ENCFF783MYY,ENCFF731JBR,ENCFF988KSK,ENCFF984UBD,ENCFF968YWL,ENCFF953FJE,ENCFF901EKY,ENCFF898WYV,ENCFF991MKU,unlifted_fetal_heart_enhancers
0,chr1,1013983,1013984,1028857,G,A,.,.,1015460,.,...,4,4,4,4,3,4,4,4,4,0
1,chr1,1043733,1043734,2063004,G,T,.,.,2124407,.,...,0,0,0,0,0,0,0,0,0,0
2,chr1,1043822,1043823,2020643,G,A,.,.,2079827,.,...,0,0,0,0,0,0,0,0,0,0
3,chr1,1045278,1045279,2161460,G,T,.,.,1887239,.,...,0,0,0,0,0,0,0,0,0,0
4,chr1,1046736,1046737,1525296,G,A,.,.,985908,.,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24720,chrX,154965968,154965969,1698786,A,C,.,.,1691162,.,...,0,0,0,0,0,0,0,0,0,0
24721,chrX,154966425,154966426,811704,C,T,.,.,800291,.,...,0,0,0,0,0,0,0,0,0,0
24722,chrX,154984682,154984683,1330594,C,G,.,.,1321273,.,...,0,0,0,0,0,0,0,0,0,0
24723,chrX,154984804,154984805,1698780,C,T,.,.,1691156,.,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
master.columns[30]

'heart_right_ventricle_tissue_male_adult_40_years.promoters'

In [ ]:
heartenn_scores = pd.read_excel("/content/drive/MyDrive/New_run_with_additional_features/BED files/Genomic analyses/heartenn.xlsx", skiprows = 1, skipfooter = 2)
heartenn_scores

,Chrom,POS,Ref,Alt,Blinded.ID,case_ctrl,anno,HeartENN_score
0,chr9,126402120,GGGGAA,G,1-04465,Case,CTCF|9526,0.933
1,chr16,73217858,C,G,1-02338,Case,CTCF|9526,0.906
2,chr12,39261126,C,T,1-02969,Case,CTCF|9526,0.819
3,chr15,89500799,C,T,1-06313,Case,CTCF|9526,0.775
4,chr1,8272271,C,T,1-04466,Case,CTCF|9526,0.684
...,...,...,...,...,...,...,...,...
170158,chrX,98726826,A,G,SSC05485,Ctrl,NaN,0.000
170159,chrX,9872783,G,A,SSC00355,Ctrl,NaN,0.000
170160,chrX,98807137,A,T,SSC05841,Ctrl,NaN,0.000
170161,chrX,9905468,C,G,SSC06670,Ctrl,NaN,0.000


In [ ]:
heartenn_scores['POS_INCR'] = heartenn_scores['POS'] + 1
heartenn_scores = heartenn_scores[['Chrom', 'POS', 'POS_INCR', 'HeartENN_score']]

In [ ]:
heartenn_scores

,Chrom,POS,POS_INCR,HeartENN_score
0,chr9,126402120,126402121,0.933
1,chr16,73217858,73217859,0.906
2,chr12,39261126,39261127,0.819
3,chr15,89500799,89500800,0.775
4,chr1,8272271,8272272,0.684
...,...,...,...,...
170158,chrX,98726826,98726827,0.000
170159,chrX,9872783,9872784,0.000
170160,chrX,98807137,98807138,0.000
170161,chrX,9905468,9905469,0.000


In [ ]:
def export_to_bed_files(df, filename):
  df.to_csv(filename, sep='\t', index=False, header=False)

export_to_bed_files(heartenn_scores, filename = "/content/drive/MyDrive/New_run_with_additional_features/BED files/Genomic analyses/clean_heartenn.txt")

In [ ]:
import subprocess

# official code to run and install the bed files, within a folder
mastersheet = "/content/drive/MyDrive/New_run_with_additional_features/mastersheet/2023_07_21_master_SCREEN_zebrafirsh_ncer_tfbs_dickelscore_vistaheartenhancers_cvdkp_hic_pchic_atac.txt"

def read_cres_files(file_lst, folder, subject, header):
  for bed_file in file_lst:
    try:
      df = pd.read_csv(os.path.join(folder, bed_file), sep = '\t', header = header)
    except pd.errors.EmptyDataError:
      # handle empty file
      print(f"One missing file: {bed_file}")
      pass
    else:
      name = bed_file[: -4]
      # print(df)
      # print(df['name'].value_counts())
      # print(bed_file)
      output_file = f"/content/drive/MyDrive/New_run_with_additional_features/BED files merging test/{subject}/intersect_{name}.txt"

      # Run the Bedtools intersect command
      command = ['bedtools', 'intersect', '-loj', '-a', mastersheet, '-b', os.path.join(folder, bed_file)]
      result = subprocess.run(command, capture_output=True, text=True)

      # Save the intersected result to the output file
      with open(output_file, 'w') as f:
          f.write(result.stdout)

      # Display the output file name
      print(f"Intersection saved as {output_file}")


def read_tfbs_files(subject, header = None):
  folder = f"/content/drive/MyDrive/New_run_with_additional_features/BED files/{subject}"

  files = [ele for ele in os.listdir(folder)]
  # files = [ele for ele in os.listdir(folder) if 'unmapped' not in ele]

  print(len(files), files)

  read_cres_files(files, folder, subject, header)

read_tfbs_files("Genomic analyses clean heartenn scores")

1 ['clean_heartenn_scores.txt']
Intersection saved as /content/drive/MyDrive/New_run_with_additional_features/BED files merging test/Genomic analyses clean heartenn scores/intersect_clean_heartenn_scores.txt


In [ ]:
sample = pd.read_csv("/content/drive/MyDrive/New_run_with_additional_features/BED files merging test/Genomic analyses clean heartenn scores/intersect_clean_heartenn_scores.txt", sep = '\t', header = None)
sample

<ipython-input-29-dd1818473f2b>:1: DtypeWarning: Columns (27,72) have mixed types. Specify dtype option on import or set low_memory=False.
  sample = pd.read_csv("/content/drive/MyDrive/New_run_with_additional_features/BED files merging test/Genomic analyses clean heartenn scores/intersect_clean_heartenn_scores.txt", sep = '\t', header = None)


,0,1,2,3,4,5,6,7,8,9,...,216,217,218,219,220,221,222,223,224,225
0,chr1,1013983,1013984,1028857,G,A,.,.,1015460,.,...,4,3,4,4,4,4,.,-1,-1,.
1,chr1,1043733,1043734,2063004,G,T,.,.,2124407,.,...,0,0,0,0,0,0,.,-1,-1,.
2,chr1,1043822,1043823,2020643,G,A,.,.,2079827,.,...,0,0,0,0,0,0,.,-1,-1,.
3,chr1,1045278,1045279,2161460,G,T,.,.,1887239,.,...,0,0,0,0,0,0,.,-1,-1,.
4,chr1,1046736,1046737,1525296,G,A,.,.,985908,.,...,0,0,0,0,0,0,.,-1,-1,.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24720,chrX,154965968,154965969,1698786,A,C,.,.,1691162,.,...,0,0,0,0,0,0,.,-1,-1,.
24721,chrX,154966425,154966426,811704,C,T,.,.,800291,.,...,0,0,0,0,0,0,.,-1,-1,.
24722,chrX,154984682,154984683,1330594,C,G,.,.,1321273,.,...,0,0,0,0,0,0,.,-1,-1,.
24723,chrX,154984804,154984805,1698780,C,T,.,.,1691156,.,...,0,0,0,0,0,0,.,-1,-1,.


In [ ]:
sample[sample[225] == '0.0529']

,0,1,2,3,4,5,6,7,8,9,...,216,217,218,219,220,221,222,223,224,225
18982,chr17,31181491,31181492,1409345,T,A,.,.,1340416,.,...,0,0,0,0,0,0,chr17,31181491,31181492,0.0529
18983,chr17,31181491,31181492,1048732,T,C,.,.,1036768,.,...,0,0,0,0,0,0,chr17,31181491,31181492,0.0529


In [ ]:
# import pandas as pd
# data = {
#     'chrom': ['chr12', 'chr15', 'chr16', 'chr20', 'chr20', 'chr7',],
#     'POS': [114704514, 90735064, 57302898, 1306044, 42780500, 4835971],
#     'POS_INCR': [114704515, 90735065, 57302899, 1306045, 42780501, 4835972]
# }

# # Create DataFrame from the dictionary
# df = pd.DataFrame(data)

# print(df)

   chrom        POS   POS_INCR
0  chr12  114704514  114704515
1  chr15   90735064   90735065
2  chr16   57302898   57302899
3  chr20    1306044    1306045
4  chr20   42780500   42780501
5   chr7    4835971    4835972


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/New_run_with_additional_features/Validation/validation_second_data.csv")
df['POS_INCR'] = df['POS'] + 1
df['chrom'] = 'chr' + df['chrom']
df = df.drop_duplicates()
df

,chrom,POS,POS_INCR
0,chr5,129647896,129647897
1,chr15,96334604,96334605
2,chr18,47851273,47851274
3,chr5,173233210,173233211
4,chr8,11755135,11755136
...,...,...,...
61,chr19,11100221,11100222
62,chr11,47348542,47348543
63,chr14,23431584,23431585
64,chr11,47346372,47346373


In [ ]:
df.to_csv("Validation/validation.txt", sep = '\t', index = False)


In [ ]:
import subprocess
validation_file = "Validation/validation.txt"
validation = pd.read_csv(validation_file, sep = '\t')
validation

,chrom,POS,POS_INCR
0,chr5,129647896,129647897
1,chr15,96334604,96334605
2,chr18,47851273,47851274
3,chr5,173233210,173233211
4,chr8,11755135,11755136
...,...,...,...
58,chr19,11100221,11100222
59,chr11,47348542,47348543
60,chr14,23431584,23431585
61,chr11,47346372,47346373


In [ ]:
validation[['chrom', 'POS', 'POS_INCR']].value_counts()

chrom  POS        POS_INCR 
chr1   156136122  156136123    1
chr19  11129511   11129512     1
       11105601   11105602     1
       11105602   11105603     1
       11107391   11107392     1
                              ..
chr18  47851273   47851274     1
chr19  11100221   11100222     1
       11100222   11100223     1
       11100346   11100347     1
chrX   154420736  154420737    1
Length: 63, dtype: int64

In [ ]:
validation[validation['chrom'] == 'chr19']

,chrom,POS,POS_INCR
22,chr19,11100346,11100347
23,chr19,11102662,11102663
24,chr19,11102663,11102664
25,chr19,11105602,11105603
26,chr19,11107391,11107392
27,chr19,11113763,11113764
28,chr19,11116093,11116094
29,chr19,11116858,11116859
30,chr19,11116999,11117000
31,chr19,11117000,11117001


In [ ]:
validation[validation['chrom'] == 'chrX ']

,chrom,POS,POS_INCR


In [ ]:
import subprocess
validation_file = "Validation/validation.txt"

# create promoters txt files with: the first initial rows as the original dataframe
# while the most right column is the column produced by the bed file
def read_cres_files(mastersheet,file_lst, folder, subject):
  for bed_file in file_lst:
    try:
      df = pd.read_csv(os.path.join(folder, bed_file), sep = '\t', header = None)
    except pd.errors.EmptyDataError:
      # handle empty file
      print(f"One missing file: {bed_file}")
      pass
    else:
      name = bed_file[: -4]
      # print(bed_file)
      output_file = f"/content/drive/MyDrive/New_run_with_additional_features/Validation merging test/{subject}/intersect_{name}.txt"

      # Run the Bedtools intersect command
      command = ['bedtools', 'intersect', '-C', '-a', mastersheet, '-b', os.path.join(folder, bed_file)]
      result = subprocess.run(command, capture_output=True, text=True)

      # Save the intersected result to the output file
      with open(output_file, 'w') as f:
          f.write(result.stdout)

      # Display the output file name
      print(f"Intersection saved as {output_file}")


def read_bed_files(mastersheet,subject):
  folder = f"/content/drive/MyDrive/New_run_with_additional_features/BED files/{subject}"
  embryo_folder = f"/content/drive/MyDrive/New_run_with_additional_features/BED files/{subject}/Clean Embryo files"
  files = os.listdir(folder)


  promoters = [ele for ele in files if 'promoter' in ele]
  enhancers = [ele for ele in files if 'enhancer' in ele]
  # for mice, embryo is a redundant column as it is already either the
  # promtoer or enhancer
  if subject == 'Human':
    embryo = [ele for ele in os.listdir(embryo_folder) if 'embryo' in ele]
  ctcf = [ele for ele in files if 'CTCF' in ele]

  # quality check for Human: enough files generated
  # investigate_levels only applies to human dna only
  # investigate_levels(promoters, folder)
  read_cres_files(mastersheet,promoters, folder, subject)

  # investigate_levels(enhancers, folder)
  read_cres_files(mastersheet,enhancers, folder, subject)

  # investigate_levels(embryo, embryo_folder)
  if subject == 'Human':
    read_cres_files(mastersheet,embryo, embryo_folder, subject)

  # investigate_levels(ctcf, folder)
  read_cres_files(mastersheet,ctcf, folder, subject)

  print(len(promoters), len(enhancers), len(ctcf))

read_bed_files(validation_file, subject = "Human")
read_bed_files(validation_file, subject = 'Mouse')

from datetime import date
today = date.today()
import re
today = re.sub(r'-', '_', str(today))
def export_mastersheet(df, note):
  today = date.today()
  import re
  today = re.sub(r'-', '_', str(today))
  df.to_csv(f"Validation/{today}_validation_{note}.txt", index = False, sep = '\t')
  print(today)


# quality check for mastersheet before merging and run merging using bedtools

# quality check for mastersheet before merging and run merging using bedtools
def merge_based_on_3_cols(df1, df2, target_col, col_index):
  df2 = df2.rename(columns = {0:'chrom', 1: 'POS', 2: "POS_INCR", col_index: target_col})[['chrom', 'POS', 'POS_INCR', target_col]]
  return pd.merge(df1, df2, on = ['chrom', 'POS', 'POS_INCR'])


# merge() arguments: mastersheet and subject
def merge(subject, mastersheet, header = None):
  master = pd.read_csv(mastersheet, sep = '\t')
  folder = f"/content/drive/MyDrive/New_run_with_additional_features/Validation merging test/{subject}"
  print(len(os.listdir(folder)))
  for file in os.listdir(folder):
    print(file)
    df = pd.read_csv(os.path.join(folder, file), sep = '\t', header = None)
    # print(df)
    # print(df[df.columns[-1]].value_counts())
    # master[file[10:-4]] = df[df.columns[-1]]
    # print(df)
    # print(master)
    master[file[10:-4]] = df[df.columns[-1]]
    # master = merge_based_on_3_cols(master, df, file[10:-4], df.columns[-1])
    # break
  return master



One missing file: heart_left_ventricle_tissue_male_adult_34_years.promoters.bed
One missing file: heart_left_ventricle_tissue_male_child_3_years.promoters.bed
One missing file: heart_right_ventricle_tissue_male_adult_34_years.promoters.bed
One missing file: heart_right_ventricle_tissue_male_child_3_years.promoters.bed
Intersection saved as /content/drive/MyDrive/New_run_with_additional_features/Validation merging test/Human/intersect_heart_left_ventricle_tissue_female_adult_53_years.promoters.txt
Intersection saved as /content/drive/MyDrive/New_run_with_additional_features/Validation merging test/Human/intersect_heart_right_ventricle_tissue_female_adult_46_years.promoters.txt
Intersection saved as /content/drive/MyDrive/New_run_with_additional_features/Validation merging test/Human/intersect_heart_right_ventricle_tissue_male_adult_40_years.promoters.txt
One missing file: heart_left_ventricle_tissue_female_adult_51_years.enhancers.bed
One missing file: heart_left_ventricle_tissue_male_a

In [ ]:
master = merge("Human", validation_file)
export_mastersheet(master, note = 'SCREEN_human')
validation_file = f"Validation/{today}_validation_SCREEN_human.txt"
master = merge("Mouse", validation_file)
export_mastersheet(master, note = 'SCREEN_human_mouse')

34
intersect_heart_left_ventricle_tissue_female_adult_53_years.promoters.txt
intersect_heart_right_ventricle_tissue_female_adult_46_years.promoters.txt
intersect_heart_right_ventricle_tissue_female_adult_46_years.enhancers.txt
intersect_heart_left_ventricle_tissue_male_adult_40_years.enhancers.txt
intersect_heart_right_ventricle_tissue_male_adult_40_years.promoters.txt
intersect_heart_left_ventricle_tissue_female_adult_46_years.enhancers.txt
intersect_heart_right_ventricle_tissue_male_adult_40_years.enhancers.txt
intersect_heart left ventricle tissue female embryo 101 days and female embryo 103 days.txt
intersect_heart right ventricle tissue female embryo 101 days and female embryo 103 days.txt
intersect_cardiac fibroblast female embryo 94 days and female embryo 98 days.txt
intersect_heart left ventricle tissue female embryo 136 days.txt
intersect_heart tissue embryo 80 days.txt
intersect_heart tissue embryo 59 days and female embryo 76 days.txt
intersect_heart tissue embryo 101 days.t

In [ ]:
# # quality check
# df = pd.read_csv("/content/drive/MyDrive/New_run_with_additional_features/Validation/2023_08_07_validation_SCREEN_human_mouse.txt", sep = "\t")
# master = pd.read_csv(f"/content/drive/MyDrive/New_run_with_additional_features/Validation/{today}_validation_SCREEN_human_mouse.txt", sep = '\t')
# df = df[master.columns]
# print(df.equals(master))

In [ ]:
master = pd.read_csv(f"/content/drive/MyDrive/New_run_with_additional_features/Validation/{today}_validation_SCREEN_human_mouse.txt", sep = '\t')
master

,chrom,POS,POS_INCR,heart_left_ventricle_tissue_female_adult_53_years.promoters,heart_right_ventricle_tissue_female_adult_46_years.promoters,heart_right_ventricle_tissue_female_adult_46_years.enhancers,heart_left_ventricle_tissue_male_adult_40_years.enhancers,heart_right_ventricle_tissue_male_adult_40_years.promoters,heart_left_ventricle_tissue_female_adult_46_years.enhancers,heart_right_ventricle_tissue_male_adult_40_years.enhancers,...,heart_left_ventricle_tissue_female_adult_46_years.CTCF-bound cCREs,heart_left_ventricle_tissue_male_adult_40_years.CTCF-bound cCREs,C57BL-6_heart_tissue_embryo_10.5_days clean.enhancers,C57BL-6_heart_tissue_postnatal_0_days clean.promoters,C57BL-6_heart_tissue_male_adult_8_weeks clean.promoters,C57BL-6_heart_tissue_embryo_10.5_days clean.promoters,C57BL-6_heart_tissue_male_adult_8_weeks clean.enhancers,C57BL-6_heart_tissue_postnatal_0_days clean.enhancers,C57BL-6_heart_tissue_postnatal_0_days clean.CTCF-bound cCREs,C57BL-6_heart_tissue_male_adult_8_weeks clean.CTCF-bound cCREs
0,chr5,129647896,129647897,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,chr15,96334604,96334605,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,chr18,47851273,47851274,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,chr5,173233210,173233211,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,1,0,0
4,chr8,11755135,11755136,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,chr19,11100221,11100222,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59,chr11,47348542,47348543,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
60,chr14,23431584,23431585,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
61,chr11,47346372,47346373,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
pd.DataFrame(master[master.columns[3:37]].sum(axis = 1)).value_counts()

0     59
4      2
1      1
18     1
dtype: int64

In [ ]:
# master = pd.read_csv(f"/content/drive/MyDrive/New_run_with_additional_features/Validation/{today}_validation_SCREEN_human_mouse.txt", sep = '\t')
validation_file = f"/content/drive/MyDrive/New_run_with_additional_features/Validation/{today}_validation_SCREEN_human_mouse.txt"


def read_cres_files(mastersheet, file_lst, folder, subject):
  for bed_file in file_lst:
    try:
      df = pd.read_csv(os.path.join(folder, bed_file), sep = '\t', header = None)
    except pd.errors.EmptyDataError:
      # handle empty file
      print(f"One missing file: {bed_file}")
      pass
    else:
      name = bed_file[: -4]
      # print(bed_file)
      output_file = f"/content/drive/MyDrive/New_run_with_additional_features/Validation merging test/{subject}/intersect_{name}.txt"

      # Run the Bedtools intersect command
      command = ['bedtools', 'intersect', '-C', '-a', mastersheet, '-b', os.path.join(folder, bed_file)]
      result = subprocess.run(command, capture_output=True, text=True)

      # Save the intersected result to the output file
      with open(output_file, 'w') as f:
          f.write(result.stdout)

      # Display the output file name
      print(f"Intersection saved as {output_file}")

# quality check for mastersheet before merging and run merging using bedtools
def merge_based_on_3_cols(df1, df2, target_col, col_index):
  df2 = df2.rename(columns = {0:'chrom', 1: 'POS', 2: "POS_INCR", col_index: target_col})[['chrom', 'POS', 'POS_INCR', target_col]]
  return pd.merge(df1, df2, on = ['chrom', 'POS', 'POS_INCR'])

def read_zebrafish_files(mastersheet, subject):
  folder = f"/content/drive/MyDrive/New_run_with_additional_features/BED files/{subject}"

  files = [ele for ele in os.listdir(folder) if 'lift' in ele]
  print(files)

  read_cres_files(mastersheet, files, folder, subject)

read_zebrafish_files(validation_file, "Zebrafish")

# # problem with 07_07 txt file, the 27th column: ORIGIN has a confusion in the data types
# # 1 is presented as either 1 or '1'

# quality check for mastersheet before merging and run merging using bedtools
def merge_based_on_3_cols(df1, df2, target_col, col_index):
  df2 = df2.rename(columns = {0:'chrom', 1: 'POS', 2: "POS_INCR", col_index: target_col})[['chrom', 'POS', 'POS_INCR', target_col]]
  return pd.merge(df1, df2, on = ['chrom', 'POS', 'POS_INCR'])


# merge() arguments: mastersheet and subject
def merge(subject, mastersheet, header = None):
  master = pd.read_csv(mastersheet, sep = '\t')
  folder = f"/content/drive/MyDrive/New_run_with_additional_features/Validation merging test/{subject}"
  print(len(os.listdir(folder)))
  for file in os.listdir(folder):
    print(file)
    df = pd.read_csv(os.path.join(folder, file), sep = '\t', header = None)
    # print(df)
    # print(df[df.columns[-1]].value_counts())
    # master[file[10:-4]] = df[df.columns[-1]]
    # print(df)
    # print(master)
    # master[file[10:-4]] = df[df.columns[-1]]
    master = merge_based_on_3_cols(master, df, file[10:-4], df.columns[-1])
    # break
  return master


master = merge("Zebrafish", validation_file)
master = master.rename({'zebrafish_lift': 'Zebrafish'}, axis = 1)
export_mastersheet(master, note = 'SCREEN_human_mouse_zebrafish')


['zebrafish_lift.bed']
Intersection saved as /content/drive/MyDrive/New_run_with_additional_features/Validation merging test/Zebrafish/intersect_zebrafish_lift.txt
1
intersect_zebrafish_lift.txt
2023_08_22


In [ ]:
# # quality check
# df = pd.read_csv("/content/drive/MyDrive/New_run_with_additional_features/Validation/2023_08_07_validation_SCREEN_human_mouse_zebrafish.txt", sep = "\t")
# master = pd.read_csv(f"/content/drive/MyDrive/New_run_with_additional_features/Validation/{today}_validation_SCREEN_human_mouse_zebrafish.txt", sep = '\t')
# df = df[master.columns]
# print(df.equals(master))

In [ ]:
master = pd.read_csv(f"/content/drive/MyDrive/New_run_with_additional_features/Validation/{today}_validation_SCREEN_human_mouse_zebrafish.txt", sep = '\t')
master

,chrom,POS,POS_INCR,heart_left_ventricle_tissue_female_adult_53_years.promoters,heart_right_ventricle_tissue_female_adult_46_years.promoters,heart_right_ventricle_tissue_female_adult_46_years.enhancers,heart_left_ventricle_tissue_male_adult_40_years.enhancers,heart_right_ventricle_tissue_male_adult_40_years.promoters,heart_left_ventricle_tissue_female_adult_46_years.enhancers,heart_right_ventricle_tissue_male_adult_40_years.enhancers,...,heart_left_ventricle_tissue_male_adult_40_years.CTCF-bound cCREs,C57BL-6_heart_tissue_embryo_10.5_days clean.enhancers,C57BL-6_heart_tissue_postnatal_0_days clean.promoters,C57BL-6_heart_tissue_male_adult_8_weeks clean.promoters,C57BL-6_heart_tissue_embryo_10.5_days clean.promoters,C57BL-6_heart_tissue_male_adult_8_weeks clean.enhancers,C57BL-6_heart_tissue_postnatal_0_days clean.enhancers,C57BL-6_heart_tissue_postnatal_0_days clean.CTCF-bound cCREs,C57BL-6_heart_tissue_male_adult_8_weeks clean.CTCF-bound cCREs,Zebrafish
0,chr5,129647896,129647897,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,chr15,96334604,96334605,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,chr18,47851273,47851274,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,chr5,173233210,173233211,0,0,0,0,0,0,0,...,0,1,0,0,0,1,1,0,0,0
4,chr8,11755135,11755136,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,chr19,11100221,11100222,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59,chr11,47348542,47348543,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
60,chr14,23431584,23431585,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
61,chr11,47346372,47346373,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
master[['chrom', 'POS', 'POS_INCR']].value_counts()

chrom  POS        POS_INCR 
chr1   156136122  156136123    1
chr19  11129511   11129512     1
       11105601   11105602     1
       11105602   11105603     1
       11107391   11107392     1
                              ..
chr18  47851273   47851274     1
chr19  11100221   11100222     1
       11100222   11100223     1
       11100346   11100347     1
chrX   154420736  154420737    1
Length: 63, dtype: int64

In [ ]:

# done within the terminal
# saved as 11_07
# requires upload of ncer
ncer = pd.read_csv("/content/drive/MyDrive/New_run_with_additional_features/Validation/val_second_data.txt", sep = '\t', header = None)
master = pd.read_csv(f"/content/drive/MyDrive/New_run_with_additional_features/Validation/{today}_validation_SCREEN_human_mouse_zebrafish.txt", sep = '\t')
print(ncer.shape, master.shape)
def merge_based_on_3_cols(df1, df2, target_col, col_index):
  df2 = df2.rename(columns = {0:'chrom', 1: 'POS', 2: "POS_INCR", col_index: target_col})[['chrom', 'POS', 'POS_INCR', target_col]]
  print(df1.shape, df2.shape, df2.columns)
  return df2, pd.merge(df1, df2, on = ['chrom', 'POS', 'POS_INCR'])
ncer, master = merge_based_on_3_cols(master, ncer, "ncer", ncer.columns[-1])
# master['ncer'] = ncer[ncer.columns[-1]]
# export_mastersheet(master, note = 'SCREEN_human_mouse_zebrafish_ncer')
master.shape

(68, 50) (68, 46)
(68, 46) (68, 4) Index(['chrom', 'POS', 'POS_INCR', 'ncer'], dtype='object')


(104, 47)

In [ ]:
print(master[['chrom', 'POS', 'POS_INCR']].value_counts())

chrom  POS        POS_INCR 
chrX   33211281   33211282     16
chr11  47333192   47333193     16
chr19  11123345   11123346     16
chr1   156136122  156136123     1
chr19  11128086   11128087      1
       11105601   11105602      1
       11105602   11105603      1
       11107391   11107392      1
       11111513   11111514      1
       11113763   11113764      1
       11116093   11116094      1
       11116858   11116859      1
       11116999   11117000      1
       11117000   11117001      1
       11120091   11120092      1
       11120523   11120524      1
       11129511   11129512      1
       11105218   11105219      1
chr2   178542263  178542264     1
       178559309  178559310     1
chr22  20994728   20994729      1
chr3   38562414   38562415      1
       38605949   38605950      1
       38620842   38620843      1
chr5   129647896  129647897     1
chr6   7559225    7559226       1
chr7   123656237  123656238     1
       150952424  150952425     1
chrX   120447840  12

In [ ]:
ncer[ncer['chrom'] == 'chrX']

,chrom,POS,POS_INCR,ncer
2,chrX,154420736,154420737,99.093122
18,chrX,120447840,120447841,99.963385
20,chrX,33211281,33211282,99.794792
21,chrX,33211281,33211282,99.794792
22,chrX,33211281,33211282,99.794792
23,chrX,33211281,33211282,99.794792


In [ ]:
# # quality check
# df = pd.read_csv("/content/drive/MyDrive/New_run_with_additional_features/Validation/2023_08_07_validation_SCREEN_human_mouse_zebrafish_ncer.txt", sep = "\t")
# master = pd.read_csv(f"/content/drive/MyDrive/New_run_with_additional_features/Validation/{today}_validation_SCREEN_human_mouse_zebrafish_ncer.txt", sep = '\t')
# df = df[master.columns]
# print(df.equals(master))

True


In [ ]:
master = pd.read_csv(f"/content/drive/MyDrive/New_run_with_additional_features/Validation/{today}_validation_SCREEN_human_mouse_zebrafish.txt", sep = '\t')
master

,chrom,POS,POS_INCR,heart_left_ventricle_tissue_female_adult_53_years.promoters,heart_left_ventricle_tissue_female_adult_46_years.enhancers,heart_right_ventricle_tissue_male_adult_40_years.promoters,heart_left_ventricle_tissue_male_adult_40_years.enhancers,heart_right_ventricle_tissue_female_adult_46_years.promoters,heart_right_ventricle_tissue_male_adult_40_years.enhancers,heart_right_ventricle_tissue_female_adult_46_years.enhancers,...,heart_left_ventricle_tissue_male_adult_40_years.CTCF-bound cCREs,C57BL-6_heart_tissue_postnatal_0_days clean.promoters,C57BL-6_heart_tissue_postnatal_0_days clean.enhancers,C57BL-6_heart_tissue_embryo_10.5_days clean.enhancers,C57BL-6_heart_tissue_embryo_10.5_days clean.promoters,C57BL-6_heart_tissue_male_adult_8_weeks clean.enhancers,C57BL-6_heart_tissue_male_adult_8_weeks clean.promoters,C57BL-6_heart_tissue_male_adult_8_weeks clean.CTCF-bound cCREs,C57BL-6_heart_tissue_postnatal_0_days clean.CTCF-bound cCREs,Zebrafish
0,chr5,129647896,129647897,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,chr7,123656237,123656238,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,chrX,154420736,154420737,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,chr2,178542263,178542264,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,chr11,47336013,47336014,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,chr19,11100221,11100222,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
64,chr11,47348542,47348543,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
65,chr14,23431584,23431585,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
66,chr11,47346372,47346373,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
ncer

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,chr5,129647896,129647897,0,0,0,0,0,0,0,...,0,0,0,0,0,0,chr5,129647887,129647897,98.585004
1,chr7,123656237,123656238,0,0,0,0,0,0,0,...,0,0,0,0,0,0,chr7,123656236,123656246,97.037255
2,chrX,154420736,154420737,0,0,0,0,0,0,0,...,0,0,0,0,0,0,chrX,154420731,154420741,99.093122
3,chr2,178542263,178542264,0,0,0,0,0,0,0,...,0,0,0,0,0,0,chr2,178542263,178542273,98.977013
4,chr11,47336013,47336014,0,0,0,0,0,0,0,...,0,0,0,0,0,0,chr11,47336009,47336019,99.844992
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,chr19,11100221,11100222,0,0,0,0,0,0,0,...,0,0,0,0,0,0,chr19,11100214,11100224,99.605707
64,chr11,47348542,47348543,0,0,0,0,0,0,0,...,0,0,0,0,0,0,chr11,47348539,47348549,99.684180
65,chr14,23431584,23431585,0,0,0,0,0,0,0,...,0,0,0,0,0,0,chr14,23431581,23431591,99.958007
66,chr11,47346372,47346373,0,1,0,0,0,0,0,...,0,0,0,0,0,0,chr11,47346369,47346379,99.841455


In [ ]:
validation_file = f"/content/drive/MyDrive/New_run_with_additional_features/Validation/{today}_validation_SCREEN_human_mouse_zebrafish_ncer.txt"
master= pd.read_csv(validation_file, sep = '\t')
master

,chrom,POS,POS_INCR,heart_left_ventricle_tissue_female_adult_53_years.promoters,heart_left_ventricle_tissue_female_adult_46_years.enhancers,heart_right_ventricle_tissue_male_adult_40_years.promoters,heart_left_ventricle_tissue_male_adult_40_years.enhancers,heart_right_ventricle_tissue_female_adult_46_years.promoters,heart_right_ventricle_tissue_male_adult_40_years.enhancers,heart_right_ventricle_tissue_female_adult_46_years.enhancers,...,C57BL-6_heart_tissue_postnatal_0_days clean.promoters,C57BL-6_heart_tissue_postnatal_0_days clean.enhancers,C57BL-6_heart_tissue_embryo_10.5_days clean.enhancers,C57BL-6_heart_tissue_embryo_10.5_days clean.promoters,C57BL-6_heart_tissue_male_adult_8_weeks clean.enhancers,C57BL-6_heart_tissue_male_adult_8_weeks clean.promoters,C57BL-6_heart_tissue_male_adult_8_weeks clean.CTCF-bound cCREs,C57BL-6_heart_tissue_postnatal_0_days clean.CTCF-bound cCREs,Zebrafish,ncer
0,chr5,129647896,129647897,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,98.585004
1,chr7,123656237,123656238,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,97.037255
2,chrX,154420736,154420737,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,99.093122
3,chr2,178542263,178542264,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,98.977013
4,chr11,47336013,47336014,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,99.844992
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,chr19,11100221,11100222,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,99.605707
100,chr11,47348542,47348543,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,99.684180
101,chr14,23431584,23431585,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,99.958007
102,chr11,47346372,47346373,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,99.841455


In [ ]:
import os
import subprocess
# add arguments to both read functions
# Change BED files merging test to Validation merging test
validation_file = f"/content/drive/MyDrive/New_run_with_additional_features/Validation/{today}_validation_SCREEN_human_mouse_zebrafish_ncer.txt"
# validation_file = f"/content/drive/MyDrive/New_run_with_additional_features/Validation/{today}_validation_SCREEN_human_mouse.txt"


def read_cres_files(mastersheet, file_lst, folder, subject):

  for bed_file in file_lst:
    try:
      df = pd.read_csv(os.path.join(folder, bed_file), sep = '\t', header = None)
    except pd.errors.EmptyDataError:
      # handle empty file
      print(f"One missing file: {bed_file}")
      pass
    else:
      name = bed_file[: -4]
      # print(bed_file)
      output_file = f"/content/drive/MyDrive/New_run_with_additional_features/Validation merging test/{subject}/intersect_{name}.txt"

      # Run the Bedtools intersect command
      command = ['bedtools', 'intersect', '-C', '-a', mastersheet, '-b', os.path.join(folder, bed_file)]
      result = subprocess.run(command, capture_output=True, text=True)

      # Save the intersected result to the output file
      with open(output_file, 'w') as f:
          f.write(result.stdout)

      # Display the output file name
      print(f"Intersection saved as {output_file}")


def read_tfbs_files(mastersheet,subject):
  folder = f"/content/drive/MyDrive/New_run_with_additional_features/BED files/{subject}"

  files = [ele for ele in os.listdir(folder)]
  print(len(files), files)

  read_cres_files(mastersheet,files, folder, subject)

read_tfbs_files(validation_file, "TFBS")

76 ['FOXF1_pwmscan_hg38_28598_34204.bed', 'GATA4_pwmscan_hg38_28596_35342.bed', 'SOX4_pwmscan_hg38_20348_27236.bed', 'FOXH1_pwmscan_hg38_43315_3114.bed', 'HIF1A_pwmscan_hg38_15529_13200.bed', 'TBX2_pwmscan_hg38_30239_37929.bed', 'GATA3_pwmscan_hg38_11738_12391.bed', 'RARB_pwmscan_hg38_48313_3308.bed', 'FOXC2_pwmscan_hg38_30036_45203.bed', 'SOX17_pwmscan_hg38_9606_11798.bed', 'TWIST1_pwmscan_hg38_38398_47795.bed', 'MEIS1_pwmscan_hg38_27664_48032.bed', 'YY1_pwmscan_hg38_43315_7423.bed', 'MEF2D_pwmscan_hg38_11121_17332.bed', 'GLI1_pwmscan_hg38_33818_38538.bed', 'ZIC3_pwmscan_hg38_33324_35656.bed', 'PROX1_pwmscan_hg38_33324_13670.bed', 'SNAI2_pwmscan_hg38_27665_28556.bed', 'HEY2_pwmscan_hg38_43317_44258.bed', 'FOXL1_pwmscan_hg38_43316_13581.bed', 'TP53_pwmscan_hg38_4482_5454.bed', 'POU4F1_pwmscan_hg38_11435_12160.bed', 'SMAD1_pwmscan_hg38_44445_5787.bed', 'NKX3_1_pwmscan_hg38_43163_4491.bed', 'NFAC4_pwmscan_hg38_33560_42902.bed', 'SRF_pwmscan_hg38_27664_8726.bed', 'MSX1_pwmscan_hg38_37655_

In [ ]:
# quality check for mastersheet before merging and run merging using bedtools
validation_file = f"/content/drive/MyDrive/New_run_with_additional_features/Validation/{today}_validation_SCREEN_human_mouse_zebrafish_ncer.txt"
def merge_based_on_3_cols(df1, df2, target_col, col_index):
  df2 = df2.rename(columns = {0:'chrom', 1: 'POS', 2: "POS_INCR", col_index: target_col})[['chrom', 'POS', 'POS_INCR', target_col]]
  return pd.merge(df1, df2, on = ['chrom', 'POS', 'POS_INCR'])


# merge() arguments: mastersheet and subject
def merge(subject, mastersheet, header = None):
  master = pd.read_csv(mastersheet, sep = '\t')
  folder = f"/content/drive/MyDrive/New_run_with_additional_features/Validation merging test/{subject}"
  print(len(os.listdir(folder)))
  for file in os.listdir(folder):
    print(file)
    df = pd.read_csv(os.path.join(folder, file), sep = '\t', header = None)
    # print(df)
    # print(df[df.columns[-1]].value_counts())
    # master[file[10:-4]] = df[df.columns[-1]]
    # print(df)
    # print(master)
    # master[file[10:-4]] = df[df.columns[-1]]
    master = merge_based_on_3_cols(master, df, file[10:-4], df.columns[-1])
    # break
  return master

master = merge("TFBS", validation_file)

76
intersect_FOXF1_pwmscan_hg38_28598_34204.txt
intersect_GATA4_pwmscan_hg38_28596_35342.txt
intersect_SOX4_pwmscan_hg38_20348_27236.txt
intersect_FOXH1_pwmscan_hg38_43315_3114.txt
intersect_HIF1A_pwmscan_hg38_15529_13200.txt
intersect_TBX2_pwmscan_hg38_30239_37929.txt
intersect_GATA3_pwmscan_hg38_11738_12391.txt
intersect_RARB_pwmscan_hg38_48313_3308.txt
intersect_FOXC2_pwmscan_hg38_30036_45203.txt
intersect_SOX17_pwmscan_hg38_9606_11798.txt
intersect_TWIST1_pwmscan_hg38_38398_47795.txt
intersect_MEIS1_pwmscan_hg38_27664_48032.txt
intersect_YY1_pwmscan_hg38_43315_7423.txt
intersect_MEF2D_pwmscan_hg38_11121_17332.txt
intersect_GLI1_pwmscan_hg38_33818_38538.txt
intersect_ZIC3_pwmscan_hg38_33324_35656.txt
intersect_PROX1_pwmscan_hg38_33324_13670.txt
intersect_SNAI2_pwmscan_hg38_27665_28556.txt
intersect_HEY2_pwmscan_hg38_43317_44258.txt
intersect_FOXL1_pwmscan_hg38_43316_13581.txt
intersect_TP53_pwmscan_hg38_4482_5454.txt
intersect_POU4F1_pwmscan_hg38_11435_12160.txt
intersect_SMAD1_pwms

In [ ]:
# quality check
export_mastersheet(master, note = 'SCREEN_human_mouse_zebrafish_ncer_tfbs')

2023_08_11


In [ ]:
validation_file = f"/content/drive/MyDrive/New_run_with_additional_features/Validation/{today}_validation_SCREEN_human_mouse_zebrafish_ncer_tfbs.txt"
master = pd.read_csv(validation_file, sep = '\t')
master

,chrom,POS,POS_INCR,heart_left_ventricle_tissue_female_adult_53_years.promoters,heart_right_ventricle_tissue_female_adult_46_years.promoters,heart_right_ventricle_tissue_male_adult_40_years.promoters,heart_right_ventricle_tissue_female_adult_46_years.enhancers,heart_left_ventricle_tissue_male_adult_40_years.enhancers,heart_left_ventricle_tissue_female_adult_46_years.enhancers,heart_right_ventricle_tissue_male_adult_40_years.enhancers,...,ASCL1_pwmscan_hg38_29347_22951,SNAI1_pwmscan_hg38_46684_9294,SMAD3_pwmscan_hg38_47659_6575,CREB1_pwmscan_hg38_7806_15552,GLI3_pwmscan_hg38_35563_38491,ZBT14_pwmscan_hg38_43996_16382,TBX5_pwmscan_hg38_47636_48695,PAX8_pwmscan_hg38_44374_46322,GATA6_pwmscan_hg38_20055_21663,HEY1_pwmscan_hg38_28887_46496
0,chr12,114704514,114704515,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,chr15,90735064,90735065,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,chr16,57302898,57302899,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,chr20,1306044,1306045,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,chr20,42780500,42780501,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,chr7,4835971,4835972,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
import os
import subprocess
# add arguments to both read functions
# Change BED files merging test to Validation merging test
validation_file = f"/content/drive/MyDrive/New_run_with_additional_features/Validation/{today}_validation_SCREEN_human_mouse_zebrafish_ncer_tfbs.txt"


def read_cres_files(mastersheet, file_lst, folder, subject):
  for bed_file in file_lst:
    try:
      df = pd.read_csv(os.path.join(folder, bed_file), sep = '\t', header = None)
    except pd.errors.EmptyDataError:
      # handle empty file
      print(f"One missing file: {bed_file}")
      pass
    else:
      name = bed_file[: -4]
      # print(bed_file)
      output_file = f"/content/drive/MyDrive/New_run_with_additional_features/Validation merging test/{subject}/intersect_{name}.txt"

      # Run the Bedtools intersect command
      command = ['bedtools', 'intersect', '-loj', '-a', mastersheet, '-b', os.path.join(folder, bed_file)]
      result = subprocess.run(command, capture_output=True, text=True)

      # Save the intersected result to the output file
      with open(output_file, 'w') as f:
          f.write(result.stdout)

      # Display the output file name
      print(f"Intersection saved as {output_file}")


def read_tfbs_files(mastersheet,subject):
  folder = f"/content/drive/MyDrive/New_run_with_additional_features/BED files/{subject}/Unfiltered and hg38"

  files = [ele for ele in os.listdir(folder)]
  print(len(files), files)

  read_cres_files(mastersheet,files, folder, subject)

read_tfbs_files(validation_file, "Dickel")

1 ['unfiltered_hg38.bed']
Intersection saved as /content/drive/MyDrive/New_run_with_additional_features/Validation merging test/Dickel/intersect_unfiltered_hg38.txt


In [ ]:
sample = pd.read_csv("/content/drive/MyDrive/New_run_with_additional_features/Validation merging test/Dickel/intersect_unfiltered_hg38.txt", header = None, sep ='\t')
sample

,0,1,2,3,4,5,6,7,8,9,...,118,119,120,121,122,123,124,125,126,127
0,chr12,114704514,114704515,0,0,0,0,0,0,0,...,0,0,0,0,0,chr12,114703499,114704714,0.068,1
1,chr15,90735064,90735065,0,0,0,0,0,0,0,...,0,0,0,0,0,.,-1,-1,.,-1
2,chr16,57302898,57302899,0,0,0,0,0,0,0,...,0,0,0,0,0,chr16,57298445,57319101,0.375,1
3,chr20,1306044,1306045,0,0,0,0,0,0,0,...,0,0,0,0,0,.,-1,-1,.,-1
4,chr20,42780500,42780501,0,0,0,0,0,0,0,...,0,0,0,0,0,.,-1,-1,.,-1
5,chr7,4835971,4835972,0,0,0,0,0,0,0,...,0,0,0,0,0,.,-1,-1,.,-1


In [ ]:
# quality check for mastersheet before merging and run merging using bedtools
validation_file = f"/content/drive/MyDrive/New_run_with_additional_features/Validation/{today}_validation_SCREEN_human_mouse_zebrafish_ncer_tfbs.txt"
def merge_based_on_3_cols(df1, df2, target_col, col_index):
  df2 = df2.rename(columns = {0:'chrom', 1: 'POS', 2: "POS_INCR", col_index: target_col})[['chrom', 'POS', 'POS_INCR', target_col]]
  return pd.merge(df1, df2, on = ['chrom', 'POS', 'POS_INCR'])


# merge() arguments: mastersheet and subject
def merge(subject, mastersheet, header = None):
  master = pd.read_csv(mastersheet, sep = '\t')
  folder = f"/content/drive/MyDrive/New_run_with_additional_features/Validation merging test/{subject}"
  print(len(os.listdir(folder)))
  for file in os.listdir(folder):
    print(file)
    df = pd.read_csv(os.path.join(folder, file), sep = '\t', header = None)
    # print(df)
    # print(df[df.columns[-1]].value_counts())
    # master[file[10:-4]] = df[df.columns[-1]]
    # print(df)
    # print(master)
    # master[file[10:-4]] = df[df.columns[-1]]
    master = merge_based_on_3_cols(master, df, "Dickel_et_al_scoreall", df.columns[-2])
    # break
  return master

master = merge("Dickel", validation_file)

1
intersect_unfiltered_hg38.txt


In [ ]:
master

,chrom,POS,POS_INCR,heart_left_ventricle_tissue_female_adult_53_years.promoters,heart_right_ventricle_tissue_female_adult_46_years.promoters,heart_right_ventricle_tissue_male_adult_40_years.promoters,heart_right_ventricle_tissue_female_adult_46_years.enhancers,heart_left_ventricle_tissue_male_adult_40_years.enhancers,heart_left_ventricle_tissue_female_adult_46_years.enhancers,heart_right_ventricle_tissue_male_adult_40_years.enhancers,...,SNAI1_pwmscan_hg38_46684_9294,SMAD3_pwmscan_hg38_47659_6575,CREB1_pwmscan_hg38_7806_15552,GLI3_pwmscan_hg38_35563_38491,ZBT14_pwmscan_hg38_43996_16382,TBX5_pwmscan_hg38_47636_48695,PAX8_pwmscan_hg38_44374_46322,GATA6_pwmscan_hg38_20055_21663,HEY1_pwmscan_hg38_28887_46496,Dickel_et_al_scoreall
0,chr12,114704514,114704515,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.068
1,chr15,90735064,90735065,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,.
2,chr16,57302898,57302899,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.375
3,chr20,1306044,1306045,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,.
4,chr20,42780500,42780501,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,.
5,chr7,4835971,4835972,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,.


In [ ]:
export_mastersheet(master, note = 'SCREEN_human_mouse_zebrafish_ncer_tfbs_dickel')

2023_08_11


In [ ]:
validation_file = f"/content/drive/MyDrive/New_run_with_additional_features/Validation/{today}_validation_SCREEN_human_mouse_zebrafish_ncer_tfbs_dickel.txt"
master= pd.read_csv(validation_file, sep = '\t')
master

,chrom,POS,POS_INCR,heart_left_ventricle_tissue_female_adult_53_years.promoters,heart_right_ventricle_tissue_female_adult_46_years.promoters,heart_right_ventricle_tissue_male_adult_40_years.promoters,heart_right_ventricle_tissue_female_adult_46_years.enhancers,heart_left_ventricle_tissue_male_adult_40_years.enhancers,heart_left_ventricle_tissue_female_adult_46_years.enhancers,heart_right_ventricle_tissue_male_adult_40_years.enhancers,...,SNAI1_pwmscan_hg38_46684_9294,SMAD3_pwmscan_hg38_47659_6575,CREB1_pwmscan_hg38_7806_15552,GLI3_pwmscan_hg38_35563_38491,ZBT14_pwmscan_hg38_43996_16382,TBX5_pwmscan_hg38_47636_48695,PAX8_pwmscan_hg38_44374_46322,GATA6_pwmscan_hg38_20055_21663,HEY1_pwmscan_hg38_28887_46496,Dickel_et_al_scoreall
0,chr12,114704514,114704515,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.068
1,chr15,90735064,90735065,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,.
2,chr16,57302898,57302899,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.375
3,chr20,1306044,1306045,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,.
4,chr20,42780500,42780501,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,.
5,chr7,4835971,4835972,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,.


In [ ]:
validation_file = f"/content/drive/MyDrive/New_run_with_additional_features/Validation/{today}_validation_SCREEN_human_mouse_zebrafish_ncer_tfbs_dickel.txt"
master= pd.read_csv(validation_file, sep = '\t')
master

,chrom,POS,POS_INCR,heart_left_ventricle_tissue_female_adult_53_years.promoters,heart_right_ventricle_tissue_female_adult_46_years.promoters,heart_right_ventricle_tissue_male_adult_40_years.promoters,heart_right_ventricle_tissue_female_adult_46_years.enhancers,heart_left_ventricle_tissue_male_adult_40_years.enhancers,heart_left_ventricle_tissue_female_adult_46_years.enhancers,heart_right_ventricle_tissue_male_adult_40_years.enhancers,...,SNAI1_pwmscan_hg38_46684_9294,SMAD3_pwmscan_hg38_47659_6575,CREB1_pwmscan_hg38_7806_15552,GLI3_pwmscan_hg38_35563_38491,ZBT14_pwmscan_hg38_43996_16382,TBX5_pwmscan_hg38_47636_48695,PAX8_pwmscan_hg38_44374_46322,GATA6_pwmscan_hg38_20055_21663,HEY1_pwmscan_hg38_28887_46496,Dickel_et_al_scoreall
0,chr12,114704514,114704515,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.068
1,chr15,90735064,90735065,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,.
2,chr16,57302898,57302899,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.375
3,chr20,1306044,1306045,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,.
4,chr20,42780500,42780501,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,.
5,chr7,4835971,4835972,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,.


In [ ]:
import os
import subprocess
# add arguments to both read functions
# Change BED files merging test to Validation merging test
validation_file = f"/content/drive/MyDrive/New_run_with_additional_features/Validation/{today}_validation_SCREEN_human_mouse_zebrafish_ncer_tfbs_dickel.txt"


def read_cres_files(mastersheet, file_lst, folder, subject):

  for bed_file in file_lst:
    try:
      df = pd.read_csv(os.path.join(folder, bed_file), sep = '\t', header = None)
    except pd.errors.EmptyDataError:
      # handle empty file
      print(f"One missing file: {bed_file}")
      pass
    else:
      name = bed_file[: -4]
      # print(bed_file)
      output_file = f"/content/drive/MyDrive/New_run_with_additional_features/Validation merging test/{subject}/intersect_{name}.txt"

      # Run the Bedtools intersect command
      command = ['bedtools', 'intersect', '-C', '-a', mastersheet, '-b', os.path.join(folder, bed_file)]
      result = subprocess.run(command, capture_output=True, text=True)

      # Save the intersected result to the output file
      with open(output_file, 'w') as f:
          f.write(result.stdout)

      # Display the output file name
      print(f"Intersection saved as {output_file}")


def read_tfbs_files(mastersheet,subject):
  folder = f"/content/drive/MyDrive/New_run_with_additional_features/BED files/{subject}/hg38"

  files = [ele for ele in os.listdir(folder)]
  print(len(files), files)

  read_cres_files(mastersheet,files, folder, subject)

read_tfbs_files(validation_file, "Vista_human_heart_enhancers")

1 ['vista_hg38.bed']
Intersection saved as /content/drive/MyDrive/New_run_with_additional_features/Validation merging test/Vista_human_heart_enhancers/intersect_vista_hg38.txt


In [ ]:
sample = pd.read_csv("/content/drive/MyDrive/New_run_with_additional_features/Validation merging test/Vista_human_heart_enhancers/intersect_vista_hg38.txt", header = None, sep ='\t')
sample

,0,1,2,3,4,5,6,7,8,9,...,115,116,117,118,119,120,121,122,123,124
0,chr12,114704514,114704515,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.068,0
1,chr15,90735064,90735065,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,.,0
2,chr16,57302898,57302899,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.375,0
3,chr20,1306044,1306045,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,.,0
4,chr20,42780500,42780501,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,.,0
5,chr7,4835971,4835972,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,.,0


In [ ]:
# quality check for mastersheet before merging and run merging using bedtools
validation_file = f"/content/drive/MyDrive/New_run_with_additional_features/Validation/{today}_validation_SCREEN_human_mouse_zebrafish_ncer_tfbs_dickel.txt"
def merge_based_on_3_cols(df1, df2, target_col, col_index):
  df2 = df2.rename(columns = {0:'chrom', 1: 'POS', 2: "POS_INCR", col_index: target_col})[['chrom', 'POS', 'POS_INCR', target_col]]
  return pd.merge(df1, df2, on = ['chrom', 'POS', 'POS_INCR'])


# merge() arguments: mastersheet and subject
def merge(subject, mastersheet, header = None):
  master = pd.read_csv(mastersheet, sep = '\t')
  folder = f"/content/drive/MyDrive/New_run_with_additional_features/Validation merging test/{subject}"
  print(len(os.listdir(folder)))
  for file in os.listdir(folder):
    print(file)
    df = pd.read_csv(os.path.join(folder, file), sep = '\t', header = None)
    # print(df)
    # print(df[df.columns[-1]].value_counts())
    # master[file[10:-4]] = df[df.columns[-1]]
    # print(df)
    # print(master)
    # master[file[10:-4]] = df[df.columns[-1]]
    master = merge_based_on_3_cols(master, df, "VISTA_human_heart_enhancers", df.columns[-1])
    # break
  return master

master = merge("Vista_human_heart_enhancers", validation_file)

1
intersect_vista_hg38.txt


In [ ]:
export_mastersheet(master, note = 'SCREEN_human_mouse_zebrafish_ncer_tfbs_dickel_vistahumanheartenhancers')

2023_08_11


In [ ]:
validation_file = f"/content/drive/MyDrive/New_run_with_additional_features/Validation/{today}_validation_SCREEN_human_mouse_zebrafish_ncer_tfbs_dickel_vistahumanheartenhancers.txt"
master= pd.read_csv(validation_file, sep = '\t')
master

,chrom,POS,POS_INCR,heart_left_ventricle_tissue_female_adult_53_years.promoters,heart_right_ventricle_tissue_female_adult_46_years.promoters,heart_right_ventricle_tissue_male_adult_40_years.promoters,heart_right_ventricle_tissue_female_adult_46_years.enhancers,heart_left_ventricle_tissue_male_adult_40_years.enhancers,heart_left_ventricle_tissue_female_adult_46_years.enhancers,heart_right_ventricle_tissue_male_adult_40_years.enhancers,...,SMAD3_pwmscan_hg38_47659_6575,CREB1_pwmscan_hg38_7806_15552,GLI3_pwmscan_hg38_35563_38491,ZBT14_pwmscan_hg38_43996_16382,TBX5_pwmscan_hg38_47636_48695,PAX8_pwmscan_hg38_44374_46322,GATA6_pwmscan_hg38_20055_21663,HEY1_pwmscan_hg38_28887_46496,Dickel_et_al_scoreall,VISTA_human_heart_enhancers
0,chr12,114704514,114704515,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.068,0
1,chr15,90735064,90735065,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,.,0
2,chr16,57302898,57302899,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.375,0
3,chr20,1306044,1306045,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,.,0
4,chr20,42780500,42780501,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,.,0
5,chr7,4835971,4835972,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,.,0


In [ ]:
# official code to run and install the bed files, within a folder
# change the validation_file
# remember to change BED files
validation_file = f"/content/drive/MyDrive/New_run_with_additional_features/Validation/{today}_validation_SCREEN_human_mouse_zebrafish_ncer_tfbs_dickel_vistahumanheartenhancers.txt"
import subprocess
def read_cres_files(file_lst, folder, subject, header):
  for bed_file in file_lst:
    try:
      df = pd.read_csv(os.path.join(folder, bed_file), sep = '\t', header = header)
    except pd.errors.EmptyDataError:
      # handle empty file
      print(f"One missing file: {bed_file}")
      pass
    else:
      name = bed_file[: -4]
      print(df)
      print(df[df.columns[-1]].value_counts())
      # print(bed_file)
      output_file = f"/content/drive/MyDrive/New_run_with_additional_features/Validation merging test/{subject}/intersect_{name}.txt"

      # Run the Bedtools intersect command
      command = ['bedtools', 'intersect', '-C', '-a', validation_file, '-b', os.path.join(folder, bed_file)]
      result = subprocess.run(command, capture_output=True, text=True)

      # Save the intersected result to the output file
      with open(output_file, 'w') as f:
          f.write(result.stdout)

      # Display the output file name
      print(f"Intersection saved as {output_file}")


def read_tfbs_files(subject, header = None):
  folder = f"/content/drive/MyDrive/New_run_with_additional_features/BED files/{subject}"

  # files = [ele for ele in os.listdir(folder)]
  files = [ele for ele in os.listdir(folder) if 'unmapped' not in ele]

  print(len(files), files)

  read_cres_files(files, folder, subject, header)

read_tfbs_files("CVDKP/gz lifted bed files")

30 ['DFF565QGM.bed', 'DFF307LLE.bed', 'DFF316NIT.bed', 'DFF512TCX.bed', 'DFF842KEC.bed', 'DFF017NNG.bed', 'DFF444FXW.bed', 'DFF803PYP.bed', 'DFF653TBJ.bed', 'DFF371HWB.bed', 'DFF165PSK.bed', 'DFF012AGB.bed', 'DFF367ZAV.bed', 'DFF552AGR.bed', 'DFF307JWU.bed', 'DFF491GMJ.bed', 'DFF729PKJ.bed', 'DFF201AVG.bed', 'DFF249RGM.bed', 'DFF559UZT.bed', 'DFF564GOU.bed', 'DFF029APW.bed', 'DFF621YNV.bed', 'DFF208QZK.bed', 'DFF090AWG.bed', 'DFF923IAT.bed', 'DFF262NPA.bed', 'DFF975AHB.bed', 'DFF528FIC.bed', 'DFF233YXQ.bed']
           0          1          2         3
0       chr1     778640     778860  promoter
1       chr1     779100     779296  promoter
2       chr1     940380     941080  promoter
3       chr1     940851     941043  promoter
4       chr1     941028     941109  promoter
...      ...        ...        ...       ...
188965  chrX  155612200  155612360  promoter
188966  chrX  155612406  155612654  promoter
188967  chrX  155612609  155612860  promoter
188968  chrX  155612861  155613060  

In [ ]:
sample = pd.read_csv("/content/drive/MyDrive/New_run_with_additional_features/Validation merging test/CVDKP/gz lifted bed files/intersect_DFF233YXQ.txt", sep = '\t', header = None)
sample

,0,1,2,3,4,5,6,7,8,9,...,116,117,118,119,120,121,122,123,124,125
0,chr12,114704514,114704515,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.068,0,0
1,chr15,90735064,90735065,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,.,0,0
2,chr16,57302898,57302899,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.375,0,0
3,chr20,1306044,1306045,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,.,0,0
4,chr20,42780500,42780501,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,.,0,0
5,chr7,4835971,4835972,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,.,0,0


In [ ]:
# quality check for mastersheet before merging and run merging using bedtools
validation_file = f"/content/drive/MyDrive/New_run_with_additional_features/Validation/{today}_validation_SCREEN_human_mouse_zebrafish_ncer_tfbs_dickel_vistahumanheartenhancers.txt"

def merge_based_on_3_cols(df1, df2, target_col, col_index):
  df2 = df2.rename(columns = {0:'chrom', 1: 'POS', 2: "POS_INCR", col_index: target_col})[['chrom', 'POS', 'POS_INCR', target_col]]
  return pd.merge(df1, df2, on = ['chrom', 'POS', 'POS_INCR'])


# merge() arguments: mastersheet and subject
def merge(subject, mastersheet, header = None):
  master = pd.read_csv(mastersheet, sep = '\t')
  folder = f"/content/drive/MyDrive/New_run_with_additional_features/Validation merging test/{subject}"
  print(len(os.listdir(folder)))
  for file in os.listdir(folder):
    print(file)
    df = pd.read_csv(os.path.join(folder, file), sep = '\t', header = None)
    # print(df)
    # print(df[df.columns[-1]].value_counts())
    # master[file[10:-4]] = df[df.columns[-1]]
    # print(df)
    # print(master)
    # master[file[10:-4]] = df[df.columns[-1]]
    master = merge_based_on_3_cols(master, df, file[10:-4], df.columns[-1])
    # break
  return master

master = merge("CVDKP/gz lifted bed files", validation_file)

30
intersect_DFF565QGM.txt
intersect_DFF307LLE.txt
intersect_DFF316NIT.txt
intersect_DFF512TCX.txt
intersect_DFF842KEC.txt
intersect_DFF017NNG.txt
intersect_DFF444FXW.txt
intersect_DFF803PYP.txt
intersect_DFF653TBJ.txt
intersect_DFF371HWB.txt
intersect_DFF165PSK.txt
intersect_DFF012AGB.txt
intersect_DFF367ZAV.txt
intersect_DFF552AGR.txt
intersect_DFF307JWU.txt
intersect_DFF491GMJ.txt
intersect_DFF729PKJ.txt
intersect_DFF201AVG.txt
intersect_DFF249RGM.txt
intersect_DFF559UZT.txt
intersect_DFF564GOU.txt
intersect_DFF029APW.txt
intersect_DFF621YNV.txt
intersect_DFF208QZK.txt
intersect_DFF090AWG.txt
intersect_DFF923IAT.txt
intersect_DFF262NPA.txt
intersect_DFF975AHB.txt
intersect_DFF528FIC.txt
intersect_DFF233YXQ.txt


In [ ]:
master

,chrom,POS,POS_INCR,heart_left_ventricle_tissue_female_adult_53_years.promoters,heart_right_ventricle_tissue_female_adult_46_years.promoters,heart_right_ventricle_tissue_male_adult_40_years.promoters,heart_right_ventricle_tissue_female_adult_46_years.enhancers,heart_left_ventricle_tissue_male_adult_40_years.enhancers,heart_left_ventricle_tissue_female_adult_46_years.enhancers,heart_right_ventricle_tissue_male_adult_40_years.enhancers,...,DFF564GOU,DFF029APW,DFF621YNV,DFF208QZK,DFF090AWG,DFF923IAT,DFF262NPA,DFF975AHB,DFF528FIC,DFF233YXQ
0,chr12,114704514,114704515,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,chr15,90735064,90735065,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,chr16,57302898,57302899,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,chr20,1306044,1306045,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,chr20,42780500,42780501,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,chr7,4835971,4835972,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
export_mastersheet(master, note = 'SCREEN_human_mouse_zebrafish_ncer_tfbs_dickel_vistahumanheartenhancers_cvdkp')

2023_08_11


In [ ]:
# official code to run and install the bed files, within a folder
validation_file = f"/content/drive/MyDrive/New_run_with_additional_features/Validation/{today}_validation_SCREEN_human_mouse_zebrafish_ncer_tfbs_dickel_vistahumanheartenhancers_cvdkp.txt"

def read_cres_files(file_lst, folder, subject, header):
  for bed_file in file_lst:
    try:
      df = pd.read_csv(os.path.join(folder, bed_file), sep = '\t', header = header)
    except pd.errors.EmptyDataError:
      # handle empty file
      print(f"One missing file: {bed_file}")
      pass
    else:
      name = bed_file[: -4]
      # print(df)
      # print(df['name'].value_counts())
      # print(bed_file)
      output_file = f"/content/drive/MyDrive/New_run_with_additional_features/Validation merging test/{subject}/intersect_{name}.txt"

      # Run the Bedtools intersect command
      command = ['bedtools', 'intersect', '-C', '-a', validation_file, '-b', os.path.join(folder, bed_file)]
      result = subprocess.run(command, capture_output=True, text=True)

      # Save the intersected result to the output file
      with open(output_file, 'w') as f:
          f.write(result.stdout)

      # Display the output file name
      print(f"Intersection saved as {output_file}")


def read_tfbs_files(subject, header = None):
  folder = f"/content/drive/MyDrive/New_run_with_additional_features/BED files/{subject}"

  # files = [ele for ele in os.listdir(folder)]
  files = [ele for ele in os.listdir(folder) if 'unmapped' not in ele]

  print(len(files), files)

  read_cres_files(files, folder, subject, header)

read_tfbs_files("Hi-C")

2 ['CM_Merged_DI_TADs.bed', 'CP_Merged_DI_TADs.bed']
Intersection saved as /content/drive/MyDrive/New_run_with_additional_features/Validation merging test/Hi-C/intersect_CM_Merged_DI_TADs.txt
Intersection saved as /content/drive/MyDrive/New_run_with_additional_features/Validation merging test/Hi-C/intersect_CP_Merged_DI_TADs.txt


In [ ]:
# quality check between -C and -c
# arguments: mastersheet and subject
validation_file = f"/content/drive/MyDrive/New_run_with_additional_features/Validation/{today}_validation_SCREEN_human_mouse_zebrafish_ncer_tfbs_dickel_vistahumanheartenhancers_cvdkp.txt"


def merge_based_on_3_cols(df1, df2, target_col, col_index):
  df2 = df2.rename(columns = {0:'chrom', 1: 'POS', 2: "POS_INCR", col_index: target_col})[['chrom', 'POS', 'POS_INCR', target_col]]
  return pd.merge(df1, df2, on = ['chrom', 'POS', 'POS_INCR'])


# merge() arguments: mastersheet and subject
def merge(subject, mastersheet, header = None):
  master = pd.read_csv(mastersheet, sep = '\t')
  folder = f"/content/drive/MyDrive/New_run_with_additional_features/Validation merging test/{subject}"
  print(len(os.listdir(folder)))
  for file in os.listdir(folder):
    print(file)
    df = pd.read_csv(os.path.join(folder, file), sep = '\t', header = None)
    # print(df)
    # print(df[df.columns[-1]].value_counts())
    # master[file[10:-4]] = df[df.columns[-1]]
    # print(df)
    # print(master)
    # master[file[10:-4]] = df[df.columns[-1]]
    master = merge_based_on_3_cols(master, df, file[10:-4], df.columns[-1])
    # break
  return master

master = merge("Hi-C", validation_file)
master

2
intersect_CM_Merged_DI_TADs.txt
intersect_CP_Merged_DI_TADs.txt


,chrom,POS,POS_INCR,heart_left_ventricle_tissue_female_adult_53_years.promoters,heart_right_ventricle_tissue_female_adult_46_years.promoters,heart_right_ventricle_tissue_male_adult_40_years.promoters,heart_right_ventricle_tissue_female_adult_46_years.enhancers,heart_left_ventricle_tissue_male_adult_40_years.enhancers,heart_left_ventricle_tissue_female_adult_46_years.enhancers,heart_right_ventricle_tissue_male_adult_40_years.enhancers,...,DFF621YNV,DFF208QZK,DFF090AWG,DFF923IAT,DFF262NPA,DFF975AHB,DFF528FIC,DFF233YXQ,CM_Merged_DI_TADs,CP_Merged_DI_TADs
0,chr12,114704514,114704515,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
1,chr15,90735064,90735065,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
2,chr16,57302898,57302899,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,chr20,1306044,1306045,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
4,chr20,42780500,42780501,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
5,chr7,4835971,4835972,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1


In [ ]:
export_mastersheet(master, note = 'SCREEN_human_mouse_zebrafish_ncer_tfbs_dickel_vistahumanheartenhancers_cvdkp_hic')


2023_08_11


In [ ]:
# official code to run and install the bed files, within a folder
import subprocess
validation_file = "/content/drive/MyDrive/New_run_with_additional_features/Validation/2023_08_11_validation_SCREEN_human_mouse_zebrafish_ncer_tfbs_dickel_vistahumanheartenhancers_cvdkp_hic.txt"

def read_cres_files(mastersheet, file_lst, folder, subject, header):
  for bed_file in file_lst:
    try:
      df = pd.read_csv(os.path.join(folder, bed_file), sep = '\t', header = header)
    except pd.errors.EmptyDataError:
      # handle empty file
      print(f"One missing file: {bed_file}")
      pass
    else:
      name = bed_file[: -4]
      # print(df)
      # print(df['name'].value_counts())
      # print(bed_file)
      output_file = f"/content/drive/MyDrive/New_run_with_additional_features/Validation merging test/{subject}/intersect_{name}.txt"

      # Run the Bedtools intersect command
      command = ['bedtools', 'intersect', '-C', '-a', mastersheet, '-b', os.path.join(folder, bed_file)]
      result = subprocess.run(command, capture_output=True, text=True)

      # Save the intersected result to the output file
      with open(output_file, 'w') as f:
          f.write(result.stdout)

      # Display the output file name
      print(f"Intersection saved as {output_file}")


def read_tfbs_files(subject, mastersheet, header = None):
  folder = f"/content/drive/MyDrive/New_run_with_additional_features/BED files/{subject}"

  files = [ele for ele in os.listdir(folder)]
  # files = [ele for ele in os.listdir(folder) if 'unmapped' not in ele]

  print(len(files), files)

  read_cres_files(mastersheet, files, folder, subject, header)

read_tfbs_files("PCHiC", validation_file)

1 ['PCHiC_interaction_points_sorte_uniq_grch38.txt']
Intersection saved as /content/drive/MyDrive/New_run_with_additional_features/Validation merging test/PCHiC/intersect_PCHiC_interaction_points_sorte_uniq_grch38.txt


In [ ]:
# quality check for mastersheet before merging and run merging using bedtools
# validation_file = f"/content/drive/MyDrive/New_run_with_additional_features/Validation/{today}_validation_SCREEN_human_mouse_zebrafish_ncer.txt"
validation_file = "/content/drive/MyDrive/New_run_with_additional_features/Validation/2023_08_11_validation_SCREEN_human_mouse_zebrafish_ncer_tfbs_dickel_vistahumanheartenhancers_cvdkp_hic.txt"

def merge_based_on_3_cols(df1, df2, target_col, col_index):
  df2 = df2.rename(columns = {0:'chrom', 1: 'POS', 2: "POS_INCR", col_index: target_col})[['chrom', 'POS', 'POS_INCR', target_col]]
  return pd.merge(df1, df2, on = ['chrom', 'POS', 'POS_INCR'])


# merge() arguments: mastersheet and subject
def merge(subject, mastersheet, header = None):
  master = pd.read_csv(mastersheet, sep = '\t')
  folder = f"/content/drive/MyDrive/New_run_with_additional_features/Validation merging test/{subject}"
  print(len(os.listdir(folder)))
  for file in os.listdir(folder):
    print(file)
    df = pd.read_csv(os.path.join(folder, file), sep = '\t', header = None)
    # print(df)
    # print(df[df.columns[-1]].value_counts())
    # master[file[10:-4]] = df[df.columns[-1]]
    # print(df)
    # print(master)
    # master[file[10:-4]] = df[df.columns[-1]]
    master = merge_based_on_3_cols(master, df, file[10:-4], df.columns[-1])
    # break
  return master

master = merge("PCHiC", validation_file)

1
intersect_PCHiC_interaction_points_sorte_uniq_grch38.txt


In [ ]:
master

,chrom,POS,POS_INCR,heart_left_ventricle_tissue_female_adult_53_years.promoters,heart_right_ventricle_tissue_female_adult_46_years.promoters,heart_right_ventricle_tissue_male_adult_40_years.promoters,heart_right_ventricle_tissue_female_adult_46_years.enhancers,heart_left_ventricle_tissue_male_adult_40_years.enhancers,heart_left_ventricle_tissue_female_adult_46_years.enhancers,heart_right_ventricle_tissue_male_adult_40_years.enhancers,...,DFF208QZK,DFF090AWG,DFF923IAT,DFF262NPA,DFF975AHB,DFF528FIC,DFF233YXQ,CM_Merged_DI_TADs,CP_Merged_DI_TADs,PCHiC_interaction_points_sorte_uniq_grch38
0,chr12,114704514,114704515,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
1,chr15,90735064,90735065,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
2,chr16,57302898,57302899,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,chr20,1306044,1306045,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,1
4,chr20,42780500,42780501,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
5,chr7,4835971,4835972,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0


In [ ]:
from datetime import date
def export_mastersheet(df, note):
  today = date.today()
  import re
  today = re.sub(r'-', '_', str(today))
  df.to_csv(f"Validation/{today}_validation_{note}.txt", index = False, sep = '\t')
  print(today)

export_mastersheet(master, note = 'SCREEN_human_mouse_zebrafish_ncer_tfbs_dickel_vistahumanheartenhancers_cvdkp_hic_pchic')

2023_08_14


In [ ]:
import re
today = date.today()
today = re.sub(r'-', '_', str(today))
validation_file = f"/content/drive/MyDrive/New_run_with_additional_features/Validation/{today}_validation_SCREEN_human_mouse_zebrafish_ncer_tfbs_dickel_vistahumanheartenhancers_cvdkp_hic_pchic.txt"

# official code to run and install the bed files, within a folder
import subprocess
def read_cres_files(mastersheet, file_lst, folder, subject, header):
  for bed_file in file_lst:
    try:
      df = pd.read_csv(os.path.join(folder, bed_file), sep = '\t', header = header)
    except pd.errors.EmptyDataError:
      # handle empty file
      print(f"One missing file: {bed_file}")
      pass
    else:
      name = bed_file[: -4]
      # print(df)
      # print(df[df.columns[-1]].value_counts())
      # print(df)
      # print(bed_file)
      output_file = f"/content/drive/MyDrive/New_run_with_additional_features/Validation merging test/{subject}/intersect_{name}.txt"

      # Run the Bedtools intersect command
      command = ['bedtools', 'intersect', '-C', '-a', mastersheet, '-b', os.path.join(folder, bed_file)]
      result = subprocess.run(command, capture_output=True, text=True)

      # Save the intersected result to the output file
      with open(output_file, 'w') as f:
          f.write(result.stdout)

      # Display the output file name
      print(f"Intersection saved as {output_file}")


def read_tfbs_files(subject, mastersheet, header = None):
  folder = f"/content/drive/MyDrive/New_run_with_additional_features/BED files/{subject}"

  files = [ele for ele in os.listdir(folder)]
  # files = [ele for ele in os.listdir(folder) if 'unmapped' not in ele]

  print(len(files), files)

  read_cres_files(mastersheet, files, folder, subject, header)

read_tfbs_files("ATAC", validation_file)

38 ['ENCFF890NRK.bed', 'ENCFF811VNG.bed', 'ENCFF199VHV.bed', 'ENCFF898WYV.bed', 'ENCFF953FJE.bed', 'ENCFF643QLU.bed', 'ENCFF372VNL.bed', 'ENCFF567VUC.bed', 'ENCFF124JSN.bed', 'ENCFF072LFQ.bed', 'ENCFF901EKY.bed', 'ENCFF291YRH.bed', 'ENCFF525ZRG.bed', 'ENCFF179JFB.bed', 'ENCFF258HEB.bed', 'ENCFF731JBR.bed', 'ENCFF988KSK.bed', 'ENCFF303AKJ.bed', 'ENCFF830CEC.bed', 'ENCFF991MKU.bed', 'ENCFF257UAY.bed', 'ENCFF556YGF.bed', 'ENCFF783MYY.bed', 'ENCFF720GJU.bed', 'ENCFF820DKN.bed', 'ENCFF529QEX.bed', 'ENCFF237JBZ.bed', 'ENCFF315UYG.bed', 'ENCFF984UBD.bed', 'ENCFF055YGM.bed', 'ENCFF968YWL.bed', 'ENCFF718BTP.bed', 'ENCFF239PZJ.bed', 'ENCFF024FSS.bed', 'ENCFF174HPZ.bed', 'ENCFF560EBG.bed', 'ENCFF251ADS.bed', 'ENCFF792EIF.bed']
Intersection saved as /content/drive/MyDrive/New_run_with_additional_features/Validation merging test/ATAC/intersect_ENCFF890NRK.txt
Intersection saved as /content/drive/MyDrive/New_run_with_additional_features/Validation merging test/ATAC/intersect_ENCFF811VNG.txt
Intersec

In [ ]:
# quality check for mastersheet before merging and run merging using bedtools
validation_file = f"/content/drive/MyDrive/New_run_with_additional_features/Validation/{today}_validation_SCREEN_human_mouse_zebrafish_ncer_tfbs_dickel_vistahumanheartenhancers_cvdkp_hic_pchic.txt"

def merge_based_on_3_cols(df1, df2, target_col, col_index):
  df2 = df2.rename(columns = {0:'chrom', 1: 'POS', 2: "POS_INCR", col_index: target_col})[['chrom', 'POS', 'POS_INCR', target_col]]
  return pd.merge(df1, df2, on = ['chrom', 'POS', 'POS_INCR'])


# merge() arguments: mastersheet and subject
def merge(subject, mastersheet, header = None):
  master = pd.read_csv(mastersheet, sep = '\t')
  folder = f"/content/drive/MyDrive/New_run_with_additional_features/Validation merging test/{subject}"
  print(len(os.listdir(folder)))
  for file in os.listdir(folder):
    print(file)
    df = pd.read_csv(os.path.join(folder, file), sep = '\t', header = None)
    # print(df)
    # print(df[df.columns[-1]].value_counts())
    # master[file[10:-4]] = df[df.columns[-1]]
    # print(df)
    # print(master)
    # master[file[10:-4]] = df[df.columns[-1]]
    master = merge_based_on_3_cols(master, df, file[10:-4], df.columns[-1])
    # break
  return master

master = merge("ATAC", validation_file)

38
intersect_ENCFF890NRK.txt
intersect_ENCFF811VNG.txt
intersect_ENCFF199VHV.txt
intersect_ENCFF898WYV.txt
intersect_ENCFF953FJE.txt
intersect_ENCFF643QLU.txt
intersect_ENCFF372VNL.txt
intersect_ENCFF567VUC.txt
intersect_ENCFF124JSN.txt
intersect_ENCFF072LFQ.txt
intersect_ENCFF901EKY.txt
intersect_ENCFF291YRH.txt
intersect_ENCFF525ZRG.txt
intersect_ENCFF179JFB.txt
intersect_ENCFF258HEB.txt
intersect_ENCFF731JBR.txt
intersect_ENCFF988KSK.txt
intersect_ENCFF303AKJ.txt
intersect_ENCFF830CEC.txt
intersect_ENCFF991MKU.txt
intersect_ENCFF257UAY.txt
intersect_ENCFF556YGF.txt
intersect_ENCFF783MYY.txt
intersect_ENCFF720GJU.txt
intersect_ENCFF820DKN.txt
intersect_ENCFF529QEX.txt
intersect_ENCFF237JBZ.txt
intersect_ENCFF315UYG.txt
intersect_ENCFF984UBD.txt
intersect_ENCFF055YGM.txt
intersect_ENCFF968YWL.txt
intersect_ENCFF718BTP.txt
intersect_ENCFF239PZJ.txt
intersect_ENCFF024FSS.txt
intersect_ENCFF174HPZ.txt
intersect_ENCFF560EBG.txt
intersect_ENCFF251ADS.txt
intersect_ENCFF792EIF.txt


In [ ]:
master

,chrom,POS,POS_INCR,heart_left_ventricle_tissue_female_adult_53_years.promoters,heart_right_ventricle_tissue_female_adult_46_years.promoters,heart_right_ventricle_tissue_male_adult_40_years.promoters,heart_right_ventricle_tissue_female_adult_46_years.enhancers,heart_left_ventricle_tissue_male_adult_40_years.enhancers,heart_left_ventricle_tissue_female_adult_46_years.enhancers,heart_right_ventricle_tissue_male_adult_40_years.enhancers,...,ENCFF984UBD,ENCFF055YGM,ENCFF968YWL,ENCFF718BTP,ENCFF239PZJ,ENCFF024FSS,ENCFF174HPZ,ENCFF560EBG,ENCFF251ADS,ENCFF792EIF
0,chr12,114704514,114704515,0,0,0,0,0,0,0,...,1,0,1,1,0,2,0,0,1,1
1,chr15,90735064,90735065,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,chr16,57302898,57302899,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,chr20,1306044,1306045,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,chr20,42780500,42780501,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,chr7,4835971,4835972,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
export_mastersheet(master, note = 'SCREEN_human_mouse_zebrafish_ncer_tfbs_dickel_vistahumanheartenhancers_cvdkp_hic_pchic_atac')

2023_08_14


In [ ]:
import subprocess

# official code to run and install the bed files, within a folder
validation_file = f"/content/drive/MyDrive/New_run_with_additional_features/Validation/{today}_validation_SCREEN_human_mouse_zebrafish_ncer_tfbs_dickel_vistahumanheartenhancers_cvdkp_hic_pchic_atac.txt"


def read_cres_files(mastersheet, file_lst, folder, subject, header):
  for bed_file in file_lst:
    try:
      df = pd.read_csv(os.path.join(folder, bed_file), sep = '\t', header = header)
    except pd.errors.EmptyDataError:
      # handle empty file
      print(f"One missing file: {bed_file}")
      pass
    else:
      name = bed_file[: -4]
      # print(df)
      # print(df['name'].value_counts())
      # print(bed_file)
      output_file = f"/content/drive/MyDrive/New_run_with_additional_features/Validation merging test/{subject}/intersect_{name}.txt"

      # Run the Bedtools intersect command
      command = ['bedtools', 'intersect', '-C', '-a', mastersheet, '-b', os.path.join(folder, bed_file)]
      result = subprocess.run(command, capture_output=True, text=True)

      # Save the intersected result to the output file
      with open(output_file, 'w') as f:
          f.write(result.stdout)

      # Display the output file name
      print(f"Intersection saved as {output_file}")


def read_tfbs_files(mastersheet, subject, header = None):
  folder = f"/content/drive/MyDrive/New_run_with_additional_features/BED files/{subject}"

  files = [ele for ele in os.listdir(folder)]
  # files = [ele for ele in os.listdir(folder) if 'unmapped' not in ele]

  print(len(files), files)

  read_cres_files(mastersheet, files, folder, subject, header)

read_tfbs_files(validation_file, "Genomic analyses clean fetal heart enhancers")

1 ['unlifted_enhancers.txt']
Intersection saved as /content/drive/MyDrive/New_run_with_additional_features/Validation merging test/Genomic analyses clean fetal heart enhancers/intersect_unlifted_enhancers.txt


In [ ]:
# quality check for mastersheet before merging and run merging using bedtools
validation_file = f"/content/drive/MyDrive/New_run_with_additional_features/Validation/{today}_validation_SCREEN_human_mouse_zebrafish_ncer_tfbs_dickel_vistahumanheartenhancers_cvdkp_hic_pchic_atac.txt"

def merge_based_on_3_cols(df1, df2, target_col, col_index):
  df2 = df2.rename(columns = {0:'chrom', 1: 'POS', 2: "POS_INCR", col_index: target_col})[['chrom', 'POS', 'POS_INCR', target_col]]
  return pd.merge(df1, df2, on = ['chrom', 'POS', 'POS_INCR'])


# merge() arguments: mastersheet and subject
def merge(subject, mastersheet, header = None):
  master = pd.read_csv(mastersheet, sep = '\t')
  folder = f"/content/drive/MyDrive/New_run_with_additional_features/Validation merging test/{subject}"
  print(len(os.listdir(folder)))
  for file in os.listdir(folder):
    print(file)
    df = pd.read_csv(os.path.join(folder, file), sep = '\t', header = None)
    # print(df)
    # print(df[df.columns[-1]].value_counts())
    # master[file[10:-4]] = df[df.columns[-1]]
    # print(df)
    # print(master)
    # master[file[10:-4]] = df[df.columns[-1]]
    master = merge_based_on_3_cols(master, df, file[10:-4], df.columns[-1])
    # break
  return master

master = merge("Genomic analyses clean fetal heart enhancers", validation_file)

1
intersect_unlifted_enhancers.txt


In [ ]:
master = master.rename({'unlifted_enhancers': 'unlifted_fetal_heart_enhancers'}, axis = 1)
master

,chrom,POS,POS_INCR,heart_left_ventricle_tissue_female_adult_53_years.promoters,heart_right_ventricle_tissue_female_adult_46_years.promoters,heart_right_ventricle_tissue_male_adult_40_years.promoters,heart_right_ventricle_tissue_female_adult_46_years.enhancers,heart_left_ventricle_tissue_male_adult_40_years.enhancers,heart_left_ventricle_tissue_female_adult_46_years.enhancers,heart_right_ventricle_tissue_male_adult_40_years.enhancers,...,ENCFF055YGM,ENCFF968YWL,ENCFF718BTP,ENCFF239PZJ,ENCFF024FSS,ENCFF174HPZ,ENCFF560EBG,ENCFF251ADS,ENCFF792EIF,unlifted_fetal_heart_enhancers
0,chr12,114704514,114704515,0,0,0,0,0,0,0,...,0,1,1,0,2,0,0,1,1,0
1,chr15,90735064,90735065,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,chr16,57302898,57302899,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,chr20,1306044,1306045,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,chr20,42780500,42780501,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
5,chr7,4835971,4835972,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [ ]:
export_mastersheet(master, note = 'SCREEN_human_mouse_zebrafish_ncer_tfbs_dickel_vistahumanheartenhancers_cvdkp_hic_pchic_atac_fetalheartenhancers')

2023_08_14


In [ ]:
master

,chrom,POS,POS_INCR,heart_left_ventricle_tissue_female_adult_53_years.promoters,heart_right_ventricle_tissue_female_adult_46_years.promoters,heart_right_ventricle_tissue_male_adult_40_years.promoters,heart_right_ventricle_tissue_female_adult_46_years.enhancers,heart_left_ventricle_tissue_male_adult_40_years.enhancers,heart_left_ventricle_tissue_female_adult_46_years.enhancers,heart_right_ventricle_tissue_male_adult_40_years.enhancers,...,ENCFF055YGM,ENCFF968YWL,ENCFF718BTP,ENCFF239PZJ,ENCFF024FSS,ENCFF174HPZ,ENCFF560EBG,ENCFF251ADS,ENCFF792EIF,unlifted_fetal_heart_enhancers
0,chr12,114704514,114704515,0,0,0,0,0,0,0,...,0,1,1,0,2,0,0,1,1,0
1,chr15,90735064,90735065,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,chr16,57302898,57302899,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,chr20,1306044,1306045,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,chr20,42780500,42780501,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
5,chr7,4835971,4835972,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


# Merging for validation on chromosome 12

Note/Warning for merging quality check for bed files merging

## Create validation file

In [ ]:
# import pandas as pd
# data = {
#     'chrom': ['chr12', 'chr15', 'chr16', 'chr20', 'chr20', 'chr7',],
#     'POS': [114704514, 90735064, 57302898, 1306044, 42780500, 4835971],
#     'POS_INCR': [114704515, 90735065, 57302899, 1306045, 42780501, 4835972]
# }

# # Create DataFrame from the dictionary
# df = pd.DataFrame(data)

# print(df)

   chrom        POS   POS_INCR
0  chr12  114704514  114704515
1  chr15   90735064   90735065
2  chr16   57302898   57302899
3  chr20    1306044    1306045
4  chr20   42780500   42780501
5   chr7    4835971    4835972


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/New_run_with_additional_features/Validation/homo_sapiens_chr12_coordinates.txt", sep = ' ')
df = df.rename({'#CHROM': 'chrom'}, axis = 1)
df = df.drop(['ID', 'FILTER'], axis = 1)
df = df.drop_duplicates(subset=["POS"])
df

,chrom,POS
0,12,10010
1,12,10017
2,12,10020
3,12,10021
4,12,10022
...,...,...
32146509,12,133265283
32146510,12,133265284
32146511,12,133265286
32146512,12,133265292


In [ ]:
# df['FILTER'].value_counts()

.    32146514
Name: FILTER, dtype: int64

In [ ]:
df['POS'].value_counts()

10010        1
88563781     1
88563822     1
88563818     1
88563813     1
            ..
43734930     1
43734927     1
43734923     1
43734921     1
133265295    1
Name: POS, Length: 31176663, dtype: int64

In [ ]:
df['POS'].value_counts()

11859118     27
130913265    26
105378709    26
58693315     25
103068933    24
             ..
44369889      1
44369886      1
44369884      1
44369879      1
133265295     1
Name: POS, Length: 31176663, dtype: int64

In [ ]:
df.to_csv("Validation/validation_chr12.txt", sep = '\t', index = False)


## Testing for validation file

### ENCODE

In [2]:
import subprocess
validation_file = "Validation/validation_chr12.txt"
validation = pd.read_csv(validation_file, sep = '\t')
validation['POS_INCR'] = validation['POS'] + 1
# Transform the 'chrom' column
# validation['chrom'] = validation['chrom'].apply(lambda x: f'chr{x}')

In [3]:
validation['POS'].value_counts()

10010        1
88563781     1
88563822     1
88563818     1
88563813     1
            ..
43734930     1
43734927     1
43734923     1
43734921     1
133265295    1
Name: POS, Length: 31176663, dtype: int64

In [6]:
validation.shape

(31176663, 3)

In [ ]:
# validation['POS'].value_counts()
# save this validation into another txt file
validation.to_csv("Validation/validation_chr12_original.txt", sep = '\t', index = False)

In [4]:
!wc "Validation/validation_chr12_original.txt"

 31176664  93529992 758568071 Validation/validation_chr12_original.txt


In [2]:
os.getcwd()

'/tljh-data/home/jupyter-ngocduy.tran'

In [7]:
import subprocess
chromosome ='chr12'
validation_file = "Validation/validation_chr12.txt"
logging_file = f"Validation/logging_{chromosome}"
from datetime import date
today = date.today()
import re

# create promoters txt files with: the first initial rows as the original dataframe
# while the most right column is the column produced by the bed file
def read_cres_files(mastersheet,file_lst, folder, subject, start_ind):
  output_odd = "Validation merging test/output_odd.txt"
  output_even = "Validation merging test/output_even.txt"
  track_ind = start_ind
  for bed_file in file_lst:
    try:

      df = pd.read_csv(os.path.join(folder, bed_file), sep = '\t', header = None)
    except pd.errors.EmptyDataError:
      # handle empty file
      print(f"One missing file: {bed_file}")
      with open(logging_file, 'w') as fpo:
          fpo.write(f"One missing file: {bed_file}\n")
      pass
    else:
      name = bed_file[: -4]
      print(name)
      # print(bed_file)
      # Run the Bedtools intersect command
      if track_ind == 0:
        output_file = output_even
        command = ['bedtools', 'intersect', '-C', '-a', mastersheet, '-b', os.path.join(folder, bed_file)]
      elif(track_ind %2 ==0):
        output_file = output_even
        command = ['bedtools', 'intersect', '-C', '-a', output_odd, '-b', os.path.join(folder, bed_file)]

      else:
        output_file=  output_odd
        command = ['bedtools', 'intersect', '-C', '-a', output_even, '-b', os.path.join(folder, bed_file)]
      result = subprocess.run(command, capture_output=True, text=True)
      # Save the intersected result to the output file
      with open(output_file, 'w') as f:
          f.write(result.stdout)
      word_count = subprocess.run(["wc", output_file])
      print(word_count.stdout)
      overview = subprocess.run(["sed", "-n", "1p", output_file], capture_output = True, text = True)
      print(overview.stdout)

      # Display the output file name
      print(track_ind)
      print(f"Intersection saved as {name}")
      with open(logging_file, 'w') as fpo:
          fpo.write(name)
          # fpo.write(word_count.stdout)
          fpo.write(overview.stdout)
          fpo.write(str(track_ind))
      track_ind += 1

  return track_ind


def read_bed_files(mastersheet,subject, start_ind):
  folder = f"BED files/{subject}"
  embryo_folder = f"BED files/{subject}/Clean Embryo files"
  files = os.listdir(folder)

  promoters = sorted([ele for ele in files if 'promoter' in ele])
  enhancers = sorted([ele for ele in files if 'enhancer' in ele])
  # for mice, embryo is a redundant column as it is already either the
  # promtoer or enhancer
  if subject == 'Human':
    embryo = sorted([ele for ele in os.listdir(embryo_folder) if 'embryo' in ele])
  ctcf = sorted([ele for ele in files if 'CTCF' in ele])

  # quality check for Human: enough files generated
  # investigate_levels only applies to human dna only
  # investigate_levels(promoters, folder)
  ind = read_cres_files(mastersheet,promoters, folder, subject, start_ind = start_ind)
  print(f"Starting ind is: {ind}")
  # investigate_levels(enhancers, folder)
  ind = read_cres_files(mastersheet,enhancers, folder, subject, start_ind = ind)
  print(f"Starting ind is: {ind}")

  # investigate_levels(embryo, embryo_folder)
  if subject == 'Human':
    ind = read_cres_files(mastersheet,embryo, embryo_folder, subject, start_ind = ind)
    print(f"Starting ind is: {ind}")

  # investigate_levels(ctcf, folder)
  ind = read_cres_files(mastersheet,ctcf, folder, subject, start_ind = ind)
  print(f"Starting ind is: {ind}")

  print(len(promoters), len(enhancers), len(ctcf))
  return ind

bed_starting_ind = read_bed_files(validation_file, subject = "Human", start_ind = 0)
print(f"bed_starting_ind: {bed_starting_ind}")
bed_starting_ind = read_bed_files(validation_file, subject = 'Mouse', start_ind = bed_starting_ind)


heart_left_ventricle_tissue_female_adult_53_years.promoters
 31176663 124706652 820921378 Validation merging test/output_even.txt
None
chr12	10010	10011	0

0
Intersection saved as heart_left_ventricle_tissue_female_adult_53_years.promoters
One missing file: heart_left_ventricle_tissue_male_adult_34_years.promoters.bed
One missing file: heart_left_ventricle_tissue_male_child_3_years.promoters.bed
heart_right_ventricle_tissue_female_adult_46_years.promoters
 31176663 155883315 883274704 Validation merging test/output_odd.txt
None
chr12	10010	10011	0	0

1
Intersection saved as heart_right_ventricle_tissue_female_adult_46_years.promoters
One missing file: heart_right_ventricle_tissue_male_adult_34_years.promoters.bed
heart_right_ventricle_tissue_male_adult_40_years.promoters
 31176663 187059978 945628030 Validation merging test/output_even.txt
None
chr12	10010	10011	0	0	0

2
Intersection saved as heart_right_ventricle_tissue_male_adult_40_years.promoters
One missing file: heart_right_ventr

In [4]:
bed_starting_ind = 42
print(bed_starting_ind)
# order: Human - Mouse - TFBS - zebrafish - vista - cvdkp - hic - pchic -atac - genomic (fetal) - ncer - dickel

42


### Zebrafish

In [ ]:
# master = pd.read_csv(f"/content/drive/MyDrive/New_run_with_additional_features/Validation/{today}_validation_SCREEN_human_mouse.txt", sep = '\t')
# validation_file = f"/content/drive/MyDrive/New_run_with_additional_features/Validation/{today}_validation_SCREEN_human_mouse.txt"
validation_file = "Validation/validation_chr12.txt"
import subprocess
def read_cres_files(mastersheet, file_lst, folder, subject):
  for bed_file in file_lst:
    try:
      df = pd.read_csv(os.path.join(folder, bed_file), sep = '\t', header = None)
    except pd.errors.EmptyDataError:
      # handle empty file
      print(f"One missing file: {bed_file}")
      pass
    else:
      name = bed_file[: -4]
      # print(bed_file)
      output_file = f"/content/drive/MyDrive/New_run_with_additional_features/Validation merging test/{subject}/intersect_{name}.txt"

      # Run the Bedtools intersect command
      command = ['bedtools', 'intersect', '-C', '-a', mastersheet, '-b', os.path.join(folder, bed_file)]
      result = subprocess.run(command, capture_output=True, text=True)

      # Save the intersected result to the output file
      with open(output_file, 'w') as f:
          f.write(result.stdout)

      # Display the output file name
      print(f"Intersection saved as {output_file}")

# quality check for mastersheet before merging and run merging using bedtools
def merge_based_on_3_cols(df1, df2, target_col, col_index):
  df2 = df2.rename(columns = {0:'chrom', 1: 'POS', 2: "POS_INCR", col_index: target_col})[['chrom', 'POS', 'POS_INCR', target_col]]
  return pd.merge(df1, df2, on = ['chrom', 'POS', 'POS_INCR'])

def read_zebrafish_files(mastersheet, subject):
  folder = f"/content/drive/MyDrive/New_run_with_additional_features/BED files/{subject}"

  files = [ele for ele in os.listdir(folder) if 'lift' in ele]
  print(files)

  read_cres_files(mastersheet, files, folder, subject)

read_zebrafish_files(validation_file, "Zebrafish")

# # problem with 07_07 txt file, the 27th column: ORIGIN has a confusion in the data types
# # 1 is presented as either 1 or '1'

# quality check for mastersheet before merging and run merging using bedtools
def merge_based_on_3_cols(df1, df2, target_col, col_index):
  df2 = df2.rename(columns = {0:'chrom', 1: 'POS', 2: "POS_INCR", col_index: target_col})[['chrom', 'POS', 'POS_INCR', target_col]]
  return pd.merge(df1, df2, on = ['chrom', 'POS', 'POS_INCR'])


# merge() arguments: mastersheet and subject
def merge(subject, mastersheet, header = None):
  master = pd.read_csv(mastersheet, sep = '\t')
  folder = f"/content/drive/MyDrive/New_run_with_additional_features/Validation merging test/{subject}"
  print(len(os.listdir(folder)))
  for file in os.listdir(folder):
    print(file)
    df = pd.read_csv(os.path.join(folder, file), sep = '\t', header = None)
    # print(df)
    # print(df[df.columns[-1]].value_counts())
    # master[file[10:-4]] = df[df.columns[-1]]
    # print(df)
    # print(master)
    # master[file[10:-4]] = df[df.columns[-1]]
    master = merge_based_on_3_cols(master, df, file[10:-4], df.columns[-1])
    # break
  return master


# master = merge("Zebrafish", validation_file)
# master = master.rename({'zebrafish_lift': 'Zebrafish'}, axis = 1)
# export_mastersheet(master, note = 'SCREEN_human_mouse_zebrafish')


['zebrafish_lift.bed']
Intersection saved as /content/drive/MyDrive/New_run_with_additional_features/Validation merging test/Zebrafish/intersect_zebrafish_lift.txt


In [ ]:
# # quality check
# df = pd.read_csv("/content/drive/MyDrive/New_run_with_additional_features/Validation/2023_08_07_validation_SCREEN_human_mouse_zebrafish.txt", sep = "\t")
# master = pd.read_csv(f"/content/drive/MyDrive/New_run_with_additional_features/Validation/{today}_validation_SCREEN_human_mouse_zebrafish.txt", sep = '\t')
# df = df[master.columns]
# print(df.equals(master))

### ncER

In [ ]:

# done within the terminal
# saved as 11_07
# requires upload of ncer
ncer = pd.read_csv("/content/drive/MyDrive/New_run_with_additional_features/Validation/val_second_data.txt", sep = '\t', header = None)
master = pd.read_csv(f"/content/drive/MyDrive/New_run_with_additional_features/Validation/{today}_validation_SCREEN_human_mouse_zebrafish.txt", sep = '\t')
print(ncer.shape, master.shape)
def merge_based_on_3_cols(df1, df2, target_col, col_index):
  df2 = df2.rename(columns = {0:'chrom', 1: 'POS', 2: "POS_INCR", col_index: target_col})[['chrom', 'POS', 'POS_INCR', target_col]]
  print(df1.shape, df2.shape, df2.columns)
  return df2, pd.merge(df1, df2, on = ['chrom', 'POS', 'POS_INCR'])
ncer, master = merge_based_on_3_cols(master, ncer, "ncer", ncer.columns[-1])
# master['ncer'] = ncer[ncer.columns[-1]]
# export_mastersheet(master, note = 'SCREEN_human_mouse_zebrafish_ncer')
master.shape

(68, 50) (68, 46)
(68, 46) (68, 4) Index(['chrom', 'POS', 'POS_INCR', 'ncer'], dtype='object')


(104, 47)

In [ ]:
print(master[['chrom', 'POS', 'POS_INCR']].value_counts())

chrom  POS        POS_INCR 
chrX   33211281   33211282     16
chr11  47333192   47333193     16
chr19  11123345   11123346     16
chr1   156136122  156136123     1
chr19  11128086   11128087      1
       11105601   11105602      1
       11105602   11105603      1
       11107391   11107392      1
       11111513   11111514      1
       11113763   11113764      1
       11116093   11116094      1
       11116858   11116859      1
       11116999   11117000      1
       11117000   11117001      1
       11120091   11120092      1
       11120523   11120524      1
       11129511   11129512      1
       11105218   11105219      1
chr2   178542263  178542264     1
       178559309  178559310     1
chr22  20994728   20994729      1
chr3   38562414   38562415      1
       38605949   38605950      1
       38620842   38620843      1
chr5   129647896  129647897     1
chr6   7559225    7559226       1
chr7   123656237  123656238     1
       150952424  150952425     1
chrX   120447840  12

In [ ]:
ncer[ncer['chrom'] == 'chrX']

,chrom,POS,POS_INCR,ncer
2,chrX,154420736,154420737,99.093122
18,chrX,120447840,120447841,99.963385
20,chrX,33211281,33211282,99.794792
21,chrX,33211281,33211282,99.794792
22,chrX,33211281,33211282,99.794792
23,chrX,33211281,33211282,99.794792


In [ ]:
# # quality check
# df = pd.read_csv("/content/drive/MyDrive/New_run_with_additional_features/Validation/2023_08_07_validation_SCREEN_human_mouse_zebrafish_ncer.txt", sep = "\t")
# master = pd.read_csv(f"/content/drive/MyDrive/New_run_with_additional_features/Validation/{today}_validation_SCREEN_human_mouse_zebrafish_ncer.txt", sep = '\t')
# df = df[master.columns]
# print(df.equals(master))

True


### TFBS

In [1]:
# import os
# import subprocess
# import subprocess
# import numpy as np 
# import pandas as pd
# chromosome ='chr12'
# validation_file = "Validation/validation_chr12.txt"
# logging_file = f"Validation/logging_{chromosome}"
# from datetime import date
# today = date.today()
# import re

# def read_cres_files(mastersheet,file_lst, folder, start_ind, intersect):
#   output_odd = "Validation merging test/output_odd.txt"
#   output_even = "Validation merging test/output_even.txt"
#   track_ind = start_ind
#   for bed_file in file_lst:
#     try:

#       df = pd.read_csv(os.path.join(folder, bed_file), sep = '\t', header = None)
#     except pd.errors.EmptyDataError:
#       # handle empty file
#       print(f"One missing file: {bed_file}")
#       pass
#     else:
#       name = bed_file[: -4]
#       print(name)
#       # print(bed_file)
#       # Run the Bedtools intersect command
#       if track_ind == 0:
#         output_file = output_even
#         command = ['bedtools', 'intersect', intersect, '-a', mastersheet, '-b', os.path.join(folder, bed_file)]
#       elif(track_ind %2 ==0):
#         output_file = output_even
#         command = ['bedtools', 'intersect', intersect, '-a', output_odd, '-b', os.path.join(folder, bed_file)]

#       else:
#         output_file=  output_odd
#         command = ['bedtools', 'intersect', intersect, '-a', output_even, '-b', os.path.join(folder, bed_file)]
#       result = subprocess.run(command, capture_output=True, text=True)
#       # Save the intersected result to the output file
#       with open(output_file, 'w') as f:
#           f.write(result.stdout)

#       word_count = subprocess.run(["wc", output_file])
#       overview = subprocess.run(["sed", "-n", "1p", output_file], capture_output = True, text = True)
#       print(overview.stdout)
#       with open(logging_file, 'a') as fpo:
#           fpo.write(name)
#           fpo.write("\n")
#           fpo.write(overview.stdout)
#           fpo.write("\n")
#           fpo.write(str(track_ind))
#           fpo.write("\n")
          

#       # Display the output file name
#       print(f"Intersection saved as {name}")
#       track_ind += 1

#   return track_ind

# # example of subfolder: /hg38
# def read_general_files(mastersheet,subject, start_ind, sub_folder = "", intersect = '-C'):
#   folder = f"BED files/{subject}{sub_folder}"

#   if subject in ['CVDKP', 'Hi-C']:
#       files = [ele for ele in sorted(os.listdir(folder)) if 'unmapped' not in ele]
#   elif subject in ['TFBS']:
#       files = [ele for ele in sorted(os.listdir(folder)) if 'bed' in ele]
      
#   else:    
#       files = [ele for ele in sorted(os.listdir(folder))]

#   print(len(files), files)

#   # read_cres_files(mastersheet,files, folder, subject)
#   # ind = read_cres_files(mastersheet,files, folder, start_ind = start_ind, intersect = intersect)
#   # return ind
# bed_starting_ind = 42
# print(bed_starting_ind)
# # cvdkp and hi-c requires unmapped
# bed_starting_ind_tfbs = read_general_files(validation_file, "TFBS", start_ind = bed_starting_ind, intersect = '-C')
# # zebrafish is good, vista requires sub_folder, cvdkp requires subfolder, hic is fine, pchic is fine, atac is fine, genomic analysis is fine 
# # bed_starting_ind_zebra = read_general_files(validation_file, "Zebrafish", start_ind = bed_starting_ind_tfbs, intersect = '-C')

42
76 ['ASCL1_pwmscan_hg38_29347_22951.bed', 'CREB1_pwmscan_hg38_7806_15552.bed', 'EOMES_pwmscan_hg38_45327_5339.bed', 'FOXC1_pwmscan_hg38_14051_14776.bed', 'FOXC2_pwmscan_hg38_30036_45203.bed', 'FOXF1_pwmscan_hg38_28598_34204.bed', 'FOXH1_pwmscan_hg38_43315_3114.bed', 'FOXL1_pwmscan_hg38_43316_13581.bed', 'GATA3_pwmscan_hg38_11738_12391.bed', 'GATA4_pwmscan_hg38_28596_35342.bed', 'GATA5_pwmscan_hg38_9066_18689.bed', 'GATA6_pwmscan_hg38_20055_21663.bed', 'GLI1_pwmscan_hg38_33818_38538.bed', 'GLI2_pwmscan_hg38_7492_29088.bed', 'GLI3_pwmscan_hg38_35563_38491.bed', 'GRHL2_pwmscan_hg38_29347_19788.bed', 'HAND1_pwmscan_hg38_38445_41025.bed', 'HES1_pwmscan_hg38_35815_41265.bed', 'HEY1_pwmscan_hg38_28887_46496.bed', 'HEY2_pwmscan_hg38_43317_44258.bed', 'HIF1A_pwmscan_hg38_15529_13200.bed', 'ISL1_pwmscan_hg38_35815_41742.bed', 'JUN_pwmscan_hg38_38048_1878.bed', 'LBX1_pwmscan_hg38_27664_45938.bed', 'MEF2A_pwmscan_hg38_7806_11341.bed', 'MEF2C_pwmscan_hg38_43317_529.bed', 'MEF2D_pwmscan_hg38_1112

In [2]:
import os
import subprocess
import subprocess
import numpy as np 
import pandas as pd
chromosome ='chr12'
validation_file = "Validation/validation_chr12.txt"
logging_file = f"Validation/logging_{chromosome}"
from datetime import date
today = date.today()
import re

def read_cres_files(mastersheet,file_lst, folder, start_ind, intersect):
  output_odd = "Validation merging test/output_odd.txt"
  output_even = "Validation merging test/output_even.txt"
  track_ind = start_ind
  for bed_file in file_lst:
    try:

      df = pd.read_csv(os.path.join(folder, bed_file), sep = '\t', header = None)
    except pd.errors.EmptyDataError:
      # handle empty file
      print(f"One missing file: {bed_file}")
      pass
    else:
      name = bed_file[: -4]
      print(name)
      # print(bed_file)
      # Run the Bedtools intersect command
      if track_ind == 0:
        output_file = output_even
        command = ['bedtools', 'intersect', intersect, '-a', mastersheet, '-b', os.path.join(folder, bed_file)]
      elif(track_ind %2 ==0):
        output_file = output_even
        command = ['bedtools', 'intersect', intersect, '-a', output_odd, '-b', os.path.join(folder, bed_file)]

      else:
        output_file=  output_odd
        command = ['bedtools', 'intersect', intersect, '-a', output_even, '-b', os.path.join(folder, bed_file)]
      result = subprocess.run(command, capture_output=True, text=True)
      # Save the intersected result to the output file
      with open(output_file, 'w') as f:
          f.write(result.stdout)

      word_count = subprocess.run(["wc", output_file])
      overview = subprocess.run(["sed", "-n", "1p", output_file], capture_output = True, text = True)
      print(overview.stdout)
      with open(logging_file, 'a') as fpo:
          fpo.write(name)
          fpo.write("\n")
          fpo.write(overview.stdout)
          fpo.write("\n")
          fpo.write(str(track_ind))
          fpo.write("\n")
          

      # Display the output file name
      print(f"Intersection saved as {name}")
      track_ind += 1

  return track_ind

# example of subfolder: /hg38
def read_general_files(mastersheet,subject, start_ind, sub_folder = "", intersect = '-C'):
  folder = f"BED files/{subject}{sub_folder}"

  if subject in ['CVDKP', 'Hi-C']:
      files = [ele for ele in sorted(os.listdir(folder)) if 'unmapped' not in ele]
  elif subject in ['TFBS']:
      files = [ele for ele in sorted(os.listdir(folder)) if 'bed' in ele]
      
  else:    
      files = [ele for ele in sorted(os.listdir(folder))]

  print(len(files), files)

  # read_cres_files(mastersheet,files, folder, subject)
  ind = read_cres_files(mastersheet,files, folder, start_ind = start_ind, intersect = intersect)
  return ind
# cvdkp and hi-c requires unmapped
bed_starting_ind_tfbs = read_general_files(validation_file, "TFBS", start_ind = bed_starting_ind, intersect = '-C')
# zebrafish is good, vista requires sub_folder, cvdkp requires subfolder, hic is fine, pchic is fine, atac is fine, genomic analysis is fine 
# bed_starting_ind_zebra = read_general_files(validation_file, "Zebrafish", start_ind = bed_starting_ind_tfbs, intersect = '-C')

In [3]:
print(bed_starting_ind_tfbs)

118


In [ ]:
bed_starting_ind_zebra = read_general_files(validation_file, "Zebrafish", start_ind = bed_starting_ind_tfbs, intersect = '-C')
print(bed_starting_ind_zebra)

In [ ]:
bed_starting_ind_vista = read_general_files(validation_file, "Vista_human_heart_enhancers", start_ind = bed_starting_ind_zebra, 
                                            intersect = "-C", sub_folder = "/hg38")

In [4]:
# order: Human - Mouse - TFBS - zebrafish - vista - cvdkp - hic - pchic -atac - genomic (fetal) - ncer - dickel
# bed_starting_ind_zebra = read_general_files(validation_file, "Zebrafish", start_ind = bed_starting_ind_tfbs, intersect = '-C')
# print(bed_starting_ind_zebra)
# bed_starting_ind_vista = read_general_files(validation_file, "Vista_human_heart_enhancers", start_ind = bed_starting_ind_zebra, 
                                            # intersect = "-C", sub_folder = "/hg38")
print(bed_starting_ind_vista)
bed_starting_ind_cvdkp = read_general_files(validation_file, "CVDKP", start_ind = bed_starting_ind_vista, 
                                            intersect = "-C", sub_folder = "/gz lifted bed files")
# bed_starting_ind_hic = read_general_files(validation_file, "Hi-C", start_ind = bed_starting_ind_cvdkp, intersect = '-C')
# bed_starting_ind_pchic = read_general_files(validation_file, "PCHiC", start_ind = bed_starting_ind_hic, intersect = '-C')   
# bed_starting_ind_atac = read_general_files(validation_file, "ATAC", start_ind = bed_starting_ind_pchic, intersect = '-C')
# bed_starting_ind_fetal = read_general_files(validation_file, "Genomic analyses clean fetal heart enhancers", start_ind = bed_starting_ind_atac)

120
30 ['DFF012AGB.bed', 'DFF017NNG.bed', 'DFF029APW.bed', 'DFF090AWG.bed', 'DFF165PSK.bed', 'DFF201AVG.bed', 'DFF208QZK.bed', 'DFF233YXQ.bed', 'DFF249RGM.bed', 'DFF262NPA.bed', 'DFF307JWU.bed', 'DFF307LLE.bed', 'DFF316NIT.bed', 'DFF367ZAV.bed', 'DFF371HWB.bed', 'DFF444FXW.bed', 'DFF491GMJ.bed', 'DFF512TCX.bed', 'DFF528FIC.bed', 'DFF552AGR.bed', 'DFF559UZT.bed', 'DFF564GOU.bed', 'DFF565QGM.bed', 'DFF621YNV.bed', 'DFF653TBJ.bed', 'DFF729PKJ.bed', 'DFF803PYP.bed', 'DFF842KEC.bed', 'DFF923IAT.bed', 'DFF975AHB.bed']
DFF012AGB
  31176663 3865906212 8303320498 Validation merging test/output_even.txt
chr12	10010	10011	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0

Intersection saved as DFF012AGB
DFF017NNG
  31176663 3897082875 8365673824 Validation merging test/output_odd.txt
chr12	10010	10011	0	0	0	

In [8]:
print(bed_starting_ind_cvdkp) #150
bed_starting_ind_atac = read_general_files(validation_file, "ATAC", start_ind = bed_starting_ind_cvdkp, intersect = '-C')
# bed_starting_ind_atac = 188

150
38 ['ENCFF024FSS.bed', 'ENCFF055YGM.bed', 'ENCFF072LFQ.bed', 'ENCFF124JSN.bed', 'ENCFF174HPZ.bed', 'ENCFF179JFB.bed', 'ENCFF199VHV.bed', 'ENCFF237JBZ.bed', 'ENCFF239PZJ.bed', 'ENCFF251ADS.bed', 'ENCFF257UAY.bed', 'ENCFF258HEB.bed', 'ENCFF291YRH.bed', 'ENCFF303AKJ.bed', 'ENCFF315UYG.bed', 'ENCFF372VNL.bed', 'ENCFF525ZRG.bed', 'ENCFF529QEX.bed', 'ENCFF556YGF.bed', 'ENCFF560EBG.bed', 'ENCFF567VUC.bed', 'ENCFF643QLU.bed', 'ENCFF718BTP.bed', 'ENCFF720GJU.bed', 'ENCFF731JBR.bed', 'ENCFF783MYY.bed', 'ENCFF792EIF.bed', 'ENCFF811VNG.bed', 'ENCFF820DKN.bed', 'ENCFF830CEC.bed', 'ENCFF890NRK.bed', 'ENCFF898WYV.bed', 'ENCFF901EKY.bed', 'ENCFF953FJE.bed', 'ENCFF968YWL.bed', 'ENCFF984UBD.bed', 'ENCFF988KSK.bed', 'ENCFF991MKU.bed']
ENCFF024FSS
   31176663  4801206102 10173920278 Validation merging test/output_even.txt
chr12	10010	10011	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	

In [5]:
print(bed_starting_ind_atac)
bed_starting_ind_hic = read_general_files(validation_file, "Hi-C", start_ind = bed_starting_ind_atac, intersect = '-C')
bed_starting_ind_pchic = read_general_files(validation_file, "PCHiC", start_ind = bed_starting_ind_hic, intersect = '-C')  

188
2 ['CM_Merged_DI_TADs.bed', 'CP_Merged_DI_TADs.bed']
CM_Merged_DI_TADs
   31176663  5985919296 12543349749 Validation merging test/output_even.txt
chr12	10010	10011	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	1

Intersection saved as CM_Merged_DI_TADs
CP_Merged_DI_TADs
   31176663  6017095959 12605703075 Validation merging test/output_odd.txt
chr12	10010	10011	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0

In [7]:
print(bed_starting_ind_pchic)
bed_starting_ind_fetal = read_general_files(validation_file, "Genomic analyses clean fetal heart enhancers", start_ind = bed_starting_ind_pchic)
print(bed_starting_ind_fetal)

191
1 ['unlifted_enhancers.txt']
unlifted_enhancers
   31176663  6079449285 12730441707 Validation merging test/output_odd.txt
chr12	10010	10011	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	1	1	0	0

Intersection saved as unlifted_enhancers
192


In [ ]:
# for dickel and ncer
# run bedtools intersection with the chr12_encode_even.txt with ncer and dickel then use paste -d'\t' to append the last column to the dataset

In [ ]:
import subprocess

# official code to run and install the bed files, within a folder
validation_file = f"/content/drive/MyDrive/New_run_with_additional_features/Validation/{today}_validation_SCREEN_human_mouse_zebrafish_ncer_tfbs_dickel_vistahumanheartenhancers_cvdkp_hic_pchic_atac.txt"


def read_cres_files(mastersheet, file_lst, folder, subject, header):
  for bed_file in file_lst:
    try:
      df = pd.read_csv(os.path.join(folder, bed_file), sep = '\t', header = header)
    except pd.errors.EmptyDataError:
      # handle empty file
      print(f"One missing file: {bed_file}")
      pass
    else:
      name = bed_file[: -4]
      # print(df)
      # print(df['name'].value_counts())
      # print(bed_file)
      output_file = f"/content/drive/MyDrive/New_run_with_additional_features/Validation merging test/{subject}/intersect_{name}.txt"

      # Run the Bedtools intersect command
      command = ['bedtools', 'intersect', '-C', '-a', mastersheet, '-b', os.path.join(folder, bed_file)]
      result = subprocess.run(command, capture_output=True, text=True)

      # Save the intersected result to the output file
      with open(output_file, 'w') as f:
          f.write(result.stdout)

      # Display the output file name
      print(f"Intersection saved as {output_file}")


def read_tfbs_files(mastersheet, subject, header = None):
  folder = f"/content/drive/MyDrive/New_run_with_additional_features/BED files/{subject}"

  files = [ele for ele in os.listdir(folder)]
  # files = [ele for ele in os.listdir(folder) if 'unmapped' not in ele]

  print(len(files), files)

  read_cres_files(mastersheet, files, folder, subject, header)

read_tfbs_files(validation_file, "Genomic analyses clean fetal heart enhancers")

1 ['unlifted_enhancers.txt']
Intersection saved as /content/drive/MyDrive/New_run_with_additional_features/Validation merging test/Genomic analyses clean fetal heart enhancers/intersect_unlifted_enhancers.txt


In [ ]:
# quality check for mastersheet before merging and run merging using bedtools
validation_file = f"/content/drive/MyDrive/New_run_with_additional_features/Validation/{today}_validation_SCREEN_human_mouse_zebrafish_ncer_tfbs_dickel_vistahumanheartenhancers_cvdkp_hic_pchic_atac.txt"

def merge_based_on_3_cols(df1, df2, target_col, col_index):
  df2 = df2.rename(columns = {0:'chrom', 1: 'POS', 2: "POS_INCR", col_index: target_col})[['chrom', 'POS', 'POS_INCR', target_col]]
  return pd.merge(df1, df2, on = ['chrom', 'POS', 'POS_INCR'])


# merge() arguments: mastersheet and subject
def merge(subject, mastersheet, header = None):
  master = pd.read_csv(mastersheet, sep = '\t')
  folder = f"/content/drive/MyDrive/New_run_with_additional_features/Validation merging test/{subject}"
  print(len(os.listdir(folder)))
  for file in os.listdir(folder):
    print(file)
    df = pd.read_csv(os.path.join(folder, file), sep = '\t', header = None)
    # print(df)
    # print(df[df.columns[-1]].value_counts())
    # master[file[10:-4]] = df[df.columns[-1]]
    # print(df)
    # print(master)
    # master[file[10:-4]] = df[df.columns[-1]]
    master = merge_based_on_3_cols(master, df, file[10:-4], df.columns[-1])
    # break
  return master

master = merge("Genomic analyses clean fetal heart enhancers", validation_file)

1
intersect_unlifted_enhancers.txt


In [ ]:
master = master.rename({'unlifted_enhancers': 'unlifted_fetal_heart_enhancers'}, axis = 1)
master

,chrom,POS,POS_INCR,heart_left_ventricle_tissue_female_adult_53_years.promoters,heart_right_ventricle_tissue_female_adult_46_years.promoters,heart_right_ventricle_tissue_male_adult_40_years.promoters,heart_right_ventricle_tissue_female_adult_46_years.enhancers,heart_left_ventricle_tissue_male_adult_40_years.enhancers,heart_left_ventricle_tissue_female_adult_46_years.enhancers,heart_right_ventricle_tissue_male_adult_40_years.enhancers,...,ENCFF055YGM,ENCFF968YWL,ENCFF718BTP,ENCFF239PZJ,ENCFF024FSS,ENCFF174HPZ,ENCFF560EBG,ENCFF251ADS,ENCFF792EIF,unlifted_fetal_heart_enhancers
0,chr12,114704514,114704515,0,0,0,0,0,0,0,...,0,1,1,0,2,0,0,1,1,0
1,chr15,90735064,90735065,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,chr16,57302898,57302899,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,chr20,1306044,1306045,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,chr20,42780500,42780501,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
5,chr7,4835971,4835972,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [ ]:
export_mastersheet(master, note = 'SCREEN_human_mouse_zebrafish_ncer_tfbs_dickel_vistahumanheartenhancers_cvdkp_hic_pchic_atac_fetalheartenhancers')

2023_08_14


In [ ]:
master

,chrom,POS,POS_INCR,heart_left_ventricle_tissue_female_adult_53_years.promoters,heart_right_ventricle_tissue_female_adult_46_years.promoters,heart_right_ventricle_tissue_male_adult_40_years.promoters,heart_right_ventricle_tissue_female_adult_46_years.enhancers,heart_left_ventricle_tissue_male_adult_40_years.enhancers,heart_left_ventricle_tissue_female_adult_46_years.enhancers,heart_right_ventricle_tissue_male_adult_40_years.enhancers,...,ENCFF055YGM,ENCFF968YWL,ENCFF718BTP,ENCFF239PZJ,ENCFF024FSS,ENCFF174HPZ,ENCFF560EBG,ENCFF251ADS,ENCFF792EIF,unlifted_fetal_heart_enhancers
0,chr12,114704514,114704515,0,0,0,0,0,0,0,...,0,1,1,0,2,0,0,1,1,0
1,chr15,90735064,90735065,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,chr16,57302898,57302899,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,chr20,1306044,1306045,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,chr20,42780500,42780501,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
5,chr7,4835971,4835972,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
